# Plot TNG Cluster analysis
## Written by Eric Rohr

In [ ]:
### import modules
import illustris_python as il
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.patheffects as pe
import matplotlib.transforms as transforms
from matplotlib.gridspec import GridSpec
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d
import scipy.stats
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import ks_2samp, anderson_ksamp
import os
import time
import h5py
import rohr_utils as ru 
import random
import six
%matplotlib widget

plt.style.use('fullpage.mplstyle')

zs, times = ru.return_zs_costimes()
times /= 1.0e9 # [Gyr]
scales = 1. / (1.+ zs)

In [ ]:
os.chdir('/u/reric/Scripts/')
! pwd

In [ ]:
### set global variables and plotting kwargs
mass_binwidth = 0.1
time_binwidth = 0.5

NGaussSmooth = 1000

figsizewidth  = 6.902 # the textwidth in inches of MNRAS
figsizeratio = 9. / 16.
figsizeheight = figsizewidth * figsizeratio

figsizewidth_fullpage = figsizewidth
figsizeheight_fullpage = figsizewidth_fullpage * figsizeratio
figsizewidth_column = (244. / 508.) * figsizewidth
figsizeheight_column = figsizewidth_column * figsizeratio

gal_boundary = 0.15

hist_kwargs         = dict(histtype='step')
sc_kwargs           = dict(s=4**2, alpha=1.0, zorder=1.01, marker=r"$\rm{{{0:s}}}$".format('\u25EF'), linewidths=0.75)
plot_kwargs         = dict(fillstyle='none', ms=1, mew=0.75, alpha=0.1)
vline_kwargs        = dict(marker='None', ls='-', zorder=1, alpha=0.5, lw=1.0)
median_kwargs       = vline_kwargs.copy()
median_kwargs['lw'] = 2.0
text_kwargs         = dict()
axislabel_kwargs    = dict()
legend_kwargs       = dict()
one_one_kwargs      = dict(ls='-', marker='None', c='k', zorder=1.0, lw=2.5)
gal_boundary_kwargs = dict(marker='None', ls='--', c='k', zorder=0.9)

Mstarz0_key = 'Subhalo_Mstar_Rgal_z0'
M200cz0_key = 'HostGroup_M_Crit200_z0'
muz0_key    = 'muz0'
MCGasz0_key = 'SubhaloColdGasMass_z0'
MHGasz0_key = 'SubhaloHotGasMass_z0'
MTGasz0_key = 'SubhaloGasMass_z0'

Mstarz0_label   = r'$M_{\star}^{\rm sat}(z=0)\, [{\rm M_\odot}]$'
M200cz0_label   = r'$M_{\rm 200c}^{\rm host}(z=0)\, [{\rm M_\odot}]$'
muz0_label      = r'$\mu \equiv M_{\star}^{\rm sat} / M_{\rm 200c}^{\rm host} (z=0)$'
muz0_label_words = r'Mass Ratio $[\mu(z=0)]$'
MCGasz0_label   = r'$M_{\rm ColdGas}^{\rm sat}(z=0)\, [{\rm M_\odot}]$'
MHGasz0_label   = r'$M_{\rm HotGas}^{\rm sat}(z=0)\, [{\rm M_\odot}]$'
MTGasz0_label   = r'$M_{\rm TotGas}^{\rm sat}(z=0)\, [{\rm M_\odot}]$'
Nbranches_label = r'\# Satellites' 

tau0_infall_HG_key = 'CosmicTime_tau_infall_HotGas0'
tau100_infall_HG_key = 'CosmicTime_tau_infall_HotGas100'
tau0_infall_G_key = 'CosmicTime_tau_infall_Gas0'
tau100_infall_G_key = 'CosmicTime_tau_infall_Gas100'
tau0_infall_CG_key = 'CosmicTime_tau_infall_ColdGas0'
tau100_infall_CG_key = 'CosmicTime_tau_infall_ColdGas100'

tstrip_infall_HG_key = 'Tstrip_tau_infall_HotGas_tau0-tau100'
tstrip_infall_CG_key = 'Tstrip_tau_infall_ColdGas_tau0-tau100'
tstrip_infall_AG_key = 'Tstrip_tau_infall_Gas_tau0-tau100'

tau0_infall_label = r'Cosmic Time at Infall [$\tau_0$] [Gyr]'
tau100_infall_HG_label = r'Cosmic Time at $\tau_{100}^{\rm HotGas}$ [Gyr]'
tau100_infall_CG_label = r'Cosmic Time at $\tau_{100}^{\rm ColdGas}$ [Gyr]'
tau100_infall_AG_label = r'Cosmic Time at $\tau_{100}^{\rm Gas}$ [Gyr]'

tstrip_infall_HG_label = r'Hot Gas Loss [$\tau_{100}^{\rm HotGas} - \tau_{0}$] [Gyr]'
tstrip_infall_CG_label = r'Hot Gas Loss [$\tau_{100}^{\rm ColdGas} - \tau_{0}$] [Gyr]'
tstrip_infall_AG_label = r'Hot Gas Loss [$\tau_{100}^{\rm Gas} - \tau_{0}$] [Gyr]'

tau0_key = tau0_infall_HG_key
infall_key = tau0_key
tau0_label = tau0_infall_label
infall_label = tau0_label
tau100_key = tau100_infall_HG_key
tau100_label = tau100_infall_HG_label
tstrip_key = tstrip_infall_HG_key
tstrip_label = tstrip_infall_HG_label
tstrip_label_long = r'Total Hot Gas Loss Duration [$\tau_{100}^{\rm HotGas} - \tau_{0}$] [Gyr]'

d0_RPS_tot_key = 'HostCentricDistance_norm_tau_infall_HotGas0'
d0_RPS_tot_label = r'Distance to $z=0$ Host at infall $[R_{\rm 200c}^{\rm host}({\rm infall})]$'

fgas_infall_key = 'SubhaloColdGasFraction_tau_RPS_tot0'
fgas_infall_label = r'$f_{\rm ColdGas}(\tau_0)\equiv M_{\rm ColdGas}^{\rm sat}(\tau_0) / M_{\star}^{\rm sat}(\tau_0)$'
SHGM_infall_key = 'SubhaloHotGasMass_tau_infall_HotGas0'
SHGM_infall_label = r'$M_{\rm HotGas}^{\rm sat}(\tau_0)$'

Napopass_tau100_key = 'Napopass_tau_infall_HotGas100'
Napopass_tau100_label = r'Number of Orbits'

HCDnz0_key = 'HostCentricDistance_norm_z0'


direc_list = ['../Figures/TNGCluster/',
              '../Papers/Rohretal_TNGCluster_MassiveSatelliteCGM/figures/']
direc_PP ='../Figures/TNGCluster/'
#direc_list = [direc_PP]


In [ ]:
# define plotting labels and kwargs for each taudcit_key, gasz0_key combination

all_key = 'all'
clean_key = 'clean'
PP_key = 'preprocessed'
BP_key = 'backsplash_prev'
taudict_keys = [all_key, clean_key, PP_key, BP_key]

all_label = 'All'
clean_label = 'First-Infalling'
PP_label = 'Pre-Processed'
BP_label = 'Backsplash'
taudict_labels = [all_label, clean_label, PP_label, BP_label]
taudict_labels_dict = {all_key:all_label, clean_key:clean_label, PP_key:PP_label, BP_key:BP_label}

tot_key = 'tot'
gas_key = 'gasz0'
nogas_key = 'nogasz0'
gasz0_keys = [tot_key, gas_key, nogas_key]

tot_label = 'All'
gas_label = r'$M_{\rm Gas}^{\rm sat}(z=0) > 10^{9}\, {\rm M_\odot}$'
nogas_label = r'$M_{\rm Gas}^{\rm sat}(z=0) < 10^{9}\, {\rm M_\odot}$'
gasz0_labels = [tot_label, gas_label, nogas_label]
gasz0_labels_dict = {tot_key:tot_label, gas_key:gas_label, nogas_key:nogas_label}

### define some general plotting parameters for each taudict_key, gasz0_key combination
### general colorscheme:
### jellyfish: green (for green valley galaxies, and images are typically green, blue and yellow)
### nonjellyf: purple (could be starforming and blue or quiescent and red)
### inspected: black 
### gasz0: blue, because they still have gas and are likely star forming
### nogasz0: red, because they have no gas and are quenched
### tot: the same color as standard

histkwargs_clean = dict(**hist_kwargs, color='tab:green', lw=2.5, zorder=2.2)
histkwargs_pp = dict(**hist_kwargs, color='tab:purple', lw=2.0, zorder=2.1)
histkwargs_all = dict(**hist_kwargs, color='silver', lw=3.5, zorder=1.9)
histkwargs_BP = dict(**hist_kwargs, color='tab:orange')
#histkwargs_all['histtype'] = 'bar'
histkwargs_dict_temp = {clean_key:histkwargs_clean, PP_key:histkwargs_pp, all_key:histkwargs_all, BP_key:histkwargs_BP}

histkwargs_dict = {}
for taudict_key in taudict_keys:
    histkwargs_dict[taudict_key] = {}
    for gasz0_key in gasz0_keys:
        histkwargs_dict[taudict_key][gasz0_key] = histkwargs_dict_temp[taudict_key].copy()
    #histkwargs_dict[taudict_key][gas_key]['color'] = 'tab:blue'
    #histkwargs_dict[taudict_key][nogas_key]['color'] = 'tab:red'
    

cmaps_dict_temp = {clean_key:mpl.cm.get_cmap('Greens'),
                   PP_key:mpl.cm.get_cmap('Purples'),
                   all_key:mpl.cm.get_cmap('Greys'),
                   BP_key:mpl.cm.get_cmap('Oranges')}

cmaps_dict = {}
for taudict_key in taudict_keys:
    cmaps_dict[taudict_key] = {}
    for gasz0_key in gasz0_keys:
        cmaps_dict[taudict_key][gasz0_key] = cmaps_dict_temp[taudict_key].copy()
    #cmaps_dict[taudict_key][gas_key] = mpl.cm.get_cmap('Blues')
    #cmaps_dict[taudict_key][nogas_key] = mpl.cm.get_cmap('Reds')
    
    
## define histogram kwargs for binning tstrip by M200cz0, Mstarz0, or tau0_infall ###
### NB -- must have two sets of histogram kwargs for the two definitions of tstrip ###
sim = 'L680n8192TNG'
M200cz0_bins = [14.0, 14.5, 15.0, 15.5] 
Mstarz0_bins = [10.0, 10.5, 11.0, 12.0]
SCGM_infall_bins = [9.0, 9.5, 10., 10.5]
tau0_bins = [5.0, 7.5, 10.0, 12.5]
Norbit_bins = [0, 1, 2, 4]

tstrip_histkwargs_dict = {}
histkwargs_list_key = 'histkwargs_list'

# M200cz0
tstrip_histkwargs_dict[M200cz0_key] = dict(key=M200cz0_key, label=M200cz0_label,
                                           savelabel=r'm200cz0', bins=M200cz0_bins)

histkwargs_list    = [{}] * (len(M200cz0_bins) - 1)
histkwargs_list[0] = dict(**hist_kwargs, color='orangered', lw=1.5, zorder=1.05)
histkwargs_list[1] = dict(**hist_kwargs, color='red', lw=2.5, zorder=1.15)
histkwargs_list[2] = dict(**hist_kwargs, color='darkred', lw=3.5, zorder=1.25)
tstrip_histkwargs_dict[M200cz0_key][histkwargs_list_key] = histkwargs_list
    
# SCGM(infall)
histkwargs_list    = [{}] * (len(SCGM_infall_bins) - 1)
tstrip_histkwargs_dict[SHGM_infall_key] = dict(key=SHGM_infall_key, label=SHGM_infall_label,
                                               savelabel=r'SHGMinfall', bins=SCGM_infall_bins)
histkwargs_list[0] = dict(**hist_kwargs, color='darkorange', lw=2.0)
histkwargs_list[1] = dict(**hist_kwargs, color='orange', lw=1.5)
histkwargs_list[2] = dict(**hist_kwargs, color='wheat', lw=1.0)
tstrip_histkwargs_dict[SHGM_infall_key][histkwargs_list_key] = histkwargs_list

# Mstarz0
histkwargs_list = [{}] * (len(Mstarz0_bins) - 1)

tstrip_histkwargs_dict[Mstarz0_key] = dict(key=Mstarz0_key, label=Mstarz0_label,
                                           savelabel=r'mstarz0', bins=Mstarz0_bins)

histkwargs_list[0] = dict(**hist_kwargs, color='darkmagenta', lw=2.0)
histkwargs_list[1] = dict(**hist_kwargs, color='mediumpurple', lw=1.5)
histkwargs_list[2] = dict(**hist_kwargs, color='violet', lw=1.0)
tstrip_histkwargs_dict[Mstarz0_key][histkwargs_list_key] = histkwargs_list
      
# tau0_infall 
histkwargs_list  = [{}] * (len(tau0_bins) - 1)

tstrip_histkwargs_dict[tau0_key] = dict(key=tau0_key, label=tau0_label,
                                        savelabel='infalltime', bins=tau0_bins)

histkwargs_list[0] = dict(**hist_kwargs, color='darkgreen', lw=2.0)
histkwargs_list[1] = dict(**hist_kwargs, color='limegreen', lw=1.5)
histkwargs_list[2] = dict(**hist_kwargs, color='palegreen', lw=1.0)
tstrip_histkwargs_dict[tau0_key][histkwargs_list_key] = histkwargs_list

# Nubmer of completed orbits (== Number of apocenter passes)
tstrip_histkwargs_dict[Napopass_tau100_key] = {histkwargs_list_key:histkwargs_list.copy()}
tstrip_histkwargs_dict[Napopass_tau100_key]['bins'] = Norbit_bins
tstrip_histkwargs_dict[Napopass_tau100_key]['savelabel'] = 'Norbit'
tstrip_histkwargs_dict[Napopass_tau100_key]['label'] = Napopass_tau100_label
       
# add these dictionaries to the class
tstrip_histbins_dict = {tstrip_key:tstrip_histkwargs_dict}


In [ ]:
class TNGSim:
    
    def __init__(self, sim, taudict_key, gasz0_key,
                 tstrip_histbins_dict=tstrip_histbins_dict,
                 onlyz0=False):
        """
        Initialize the TNGsim class.
        sim == 'TNG50-1' 
        taudict_key == jel_key, non_key, or ins_key
        gasz0_key == tot_key, gas_key, nogas_key
        M200cz0_bins, Mstarz0_bins should each be length 4, including all bin edges
        """ 
        self.sim  = sim
        self.M200cz0_bins= M200cz0_bins
        self.Mstarz0_bins = Mstarz0_bins
        self.taudict_key = taudict_key
        self.gasz0_key = gasz0_key
        self.onlyz0 = onlyz0
        self.taudict_fname = return_taudict_fname(self)
        
        self.sim_label = 'TNG-Cluster'
        self.taudict_label = taudict_labels_dict[taudict_key]
        self.gasz0_label = gasz0_labels_dict[gasz0_key]
        
        #self.taudict_label = taudict_labels_dict[taudict_key]
        
        #self.gasz0_label = gasz0_labels_dict[gasz0_key]
                       
        # load the tau dictionary 
        self.tau_dict = load_taudict(self, self.taudict_fname)
        
        self.histkwargs = histkwargs_dict[taudict_key][gasz0_key]
        self.cmap = cmaps_dict[taudict_key][gasz0_key]
        self.plotkwargs = dict(**plot_kwargs, color=self.histkwargs['color'])
        
        sckwargs_dict = {}
        sckwargs_dict[M200cz0_key] = dict(**sc_kwargs, cmap='plasma',
                                          norm=mpl.colors.LogNorm(vmin=10.**min(M200cz0_bins), vmax=10.**(max(M200cz0_bins))))
        sckwargs_dict[Mstarz0_key] = dict(**sc_kwargs, cmap='viridis',
                                          norm=mpl.colors.LogNorm(vmin=10.**(min(Mstarz0_bins)), vmax=10.**(max(Mstarz0_bins))))
        sckwargs_dict[muz0_key]    = dict(**sc_kwargs, cmap='cubehelix_r',
                                          norm=mpl.colors.LogNorm(vmin=10.**(-5.5), vmax=10.**(-2.5)))
        self.sckwargs_dict = sckwargs_dict
        
        self.tstrip_histbins_dict = tstrip_histbins_dict
        
        self.save_label = sim + '_' + taudict_key + '_' + gasz0_key
        self.plot_label = '%s %s \n %s (%d)'%(self.sim_label, self.taudict_label, self.gasz0_label, self.tau_dict['SubfindID'].size)
        
        self.M200cz0_bins= M200cz0_bins
        
        return


def load_taudict(Sim, fname):

    result = {}
    with h5py.File('../Output/%s_subfindGRP_20230916/'%Sim.sim + fname, 'r') as f:
        group = f['Group']
        for dset_key in group.keys():
            result[dset_key] = group[dset_key][:]
        f.close()
        
    return result


def load_taudict_new(Sim, fname):

    result = {}
    with h5py.File('../Output/%s_subfindGRP/'%Sim.sim + fname, 'r') as f:
        group = f['Group']
        for dset_key in group.keys():
            result[dset_key] = group[dset_key][:]
        f.close()
        
    return result
        
    
def return_taudict_fname(Sim):
    """ 
    return the taudict filename for the given sim, assuming only satellites 
    """
    if Sim.onlyz0:
        return 'subfind_%s_tau_z0_%s_%s.hdf5'%(Sim.sim, Sim.taudict_key, Sim.gasz0_key)

    return 'subfind_%s_tau_%s_%s.hdf5'%(Sim.sim, Sim.taudict_key, Sim.gasz0_key)

    
def add_redshift_sincez2(ax, label=True):
    """
    For a given x axis, add redshift since z=2 to the top x-axis. 
    Optionally label the axis + tick marks.
    Returns ax
    """
    ticks_SnapNum = [33, 40, 50, 59, 67, 78, 84, 91, 99]
    ticks_costime = times[ticks_SnapNum]
    ticks_labels = ['2', '1.5', '1', '0.7', '0.5', '0.3', '0.2', '0.1', '0']

    xlolim = ru.floor_to_value(times[np.argmin(abs(zs - 2.5))], 0.1)
    xhilim = ru.ceil_to_value(times[np.argmin(abs(zs - 0.0))], 0.1)
    xhilim = 14.1

    ax.set_xlim(xlolim, xhilim)

    redshift_ax = ax.twiny()
    redshift_ax.set_xlim(ax.get_xlim())
    redshift_ax.set_xticks(ticks_costime)
    redshift_ax.tick_params(axis='both', which='minor', top=False)
    
    yscale = ax.get_yscale()
    if yscale == 'log':
        locmin = mpl.ticker.LogLocator(subs=(0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9))
        ax.yaxis.set_minor_locator(locmin)
        #ax.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())

    if label:
        redshift_ax.set_xlabel(r'Redshift', **axislabel_kwargs)
        redshift_ax.set_xticklabels(ticks_labels)
    else:
        redshift_ax.set_xticklabels([])
        
    return ax


def add_redshift_sincez5(ax, label=True):
    """
    For a given x axis, add redshift since z=2 to the top x-axis. 
    Optionally label the axis + tick marks.
    Returns ax
    """
    ticks_SnapNum = [17, 33, 50, 67, 84, 99]
    ticks_costime = times[ticks_SnapNum]
    ticks_labels = ['5', '2', '1', '0.5', '0.2', '0']

    xlolim = ru.floor_to_value(times[np.argmin(abs(zs - 7))], 0.1)
    xlolim = 0.9
    xhilim = ru.ceil_to_value(times[np.argmin(abs(zs - 0.0))], 0.1)
    xhilim = 14.1

    ax.set_xlim(xlolim, xhilim)

    redshift_ax = ax.twiny()
    redshift_ax.set_xlim(ax.get_xlim())
    redshift_ax.set_xticks(ticks_costime)
    redshift_ax.tick_params(axis='both', which='minor', top=False)
    
    yscale = ax.get_yscale()
    if yscale == 'log':
        locmin = mpl.ticker.LogLocator(subs=(0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9))
        ax.yaxis.set_minor_locator(locmin)
        #ax.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())

    if label:
        redshift_ax.set_xlabel(r'Redshift', **axislabel_kwargs)
        redshift_ax.set_xticklabels(ticks_labels)
    else:
        redshift_ax.set_xticklabels([])
        
    return ax




In [ ]:
direc = '../Output/L680n8192TNG_subfindGRP/'
direc_old = '../Output/L680n8192TNG_subfindGRP_20230916/' 
fname = 'subfind_L680n8192TNG_branches_z0_all.hdf5'

f = h5py.File(direc + fname, 'r')
f_keys = list(f.keys())
f_old = h5py.File(direc_old + fname, 'r')
f_old_keys = list(f_old.keys())

In [ ]:
sim = 'L680n8192TNG'
TNGCluster_all = TNGSim(sim, all_key, tot_key)
TNGCluster_all_gasz0 = TNGSim(sim, all_key, gas_key)
TNGCluster_all_nogasz0 = TNGSim(sim, all_key, nogas_key)
TNGCluster_all_tot = TNGCluster_all

TNGCluster_clean = TNGSim(sim, clean_key, tot_key)
TNGCluster_clean_gasz0 = TNGSim(sim, clean_key, gas_key)
TNGCluster_clean_nogasz0 = TNGSim(sim, clean_key, nogas_key)
TNGCluster_clean_tot = TNGCluster_clean

TNGCluster_PP = TNGSim(sim, PP_key, tot_key)
TNGCluster_PP_gasz0 = TNGSim(sim, PP_key, gas_key)
TNGCluster_PP_nogasz0 = TNGSim(sim, PP_key, nogas_key)
TNGCluster_PP_tot = TNGCluster_PP

TNGCluster_BP = TNGSim(sim, BP_key, tot_key)
TNGCluster_BP_gasz0 = TNGSim(sim, BP_key, gas_key)
TNGCluster_BP_nogasz0 = TNGSim(sim, BP_key, nogas_key)
TNGCluster_BP_tot = TNGCluster_BP

all_Sims = [TNGCluster_all, TNGCluster_clean, TNGCluster_PP,
           TNGCluster_all_gasz0, TNGCluster_all_nogasz0, 
           TNGCluster_clean_gasz0, TNGCluster_clean_nogasz0,
           TNGCluster_PP_gasz0, TNGCluster_PP_nogasz0]

In [ ]:
TNGCluster_all.tau_dict['SubfindID'].size

In [ ]:
TNGCluster_all_z0 = TNGSim(sim, all_key, tot_key, onlyz0=True)
TNGCluster_all_z0_gasz0 = TNGSim(sim, all_key, gas_key, onlyz0=True)
TNGCluster_all_z0_nogasz0 = TNGSim(sim, all_key, nogas_key, onlyz0=True)
TNGCluster_all_z0_tot = TNGCluster_all_z0

In [ ]:
TNGCluster_all_z0_gasz0.tau_dict['SubfindID'].size

In [ ]:
fgas_z0_key = 'fgas_z0'
fCgas_z0_key = 'fCgas_z0'
fHgas_z0_key = 'fHGas_z0'

SubhaloMassz0_key = 'SubhaloMass_z0'

fgas_z0_keys = [fgas_z0_key,
                fCgas_z0_key,
                fHgas_z0_key]

SubhaloGasMassz0_keys = [MTGasz0_key,
                        MCGasz0_key,
                        MHGasz0_key]

time_since_infall_key = 'TimeSinceInfall'

def add_fgas_dset(Sim):
    """
    Add dataset to tau_dict.
    """
    tau_dict = Sim.tau_dict
    SubhaloMassz0 = tau_dict[Mstarz0_key].copy()
    
    result = np.zeros(SubhaloMassz0.size, SubhaloMassz0.dtype) - 1.
    
    for i, fgas_z0_key in enumerate(fgas_z0_keys):
        dset = result.copy()
        gas_mass_key = SubhaloGasMassz0_keys[i]
        gas_mass = tau_dict[gas_mass_key].copy()
        mask = gas_mass > 0
        dset[mask] = gas_mass[mask] / (SubhaloMassz0[mask] + gas_mass[mask])
        tau_dict[fgas_z0_key] = dset
        
    return
        
def add_time_since_infall_dset(Sim):
    """
    Add dataset to tau_dict.
    """
    tau_dict = Sim.tau_dict
    tau0 = tau_dict[tau0_infall_HG_key].copy()
    result = np.zeros(tau0.size, tau0.dtype)
    indices = tau0 > 0
    time_z0 = tau_dict['CosmicTime_z0'][indices][0]
    result[indices] = time_z0 - tau0[indices]
    tau_dict[time_since_infall_key] = result
    
    return
    
Sims = [TNGCluster_all, TNGCluster_clean, TNGCluster_PP, TNGCluster_all_z0, TNGCluster_all_z0_gasz0]
for Sim in all_Sims:
    add_fgas_dset(Sim)
    add_time_since_infall_dset(Sim)




In [ ]:
tau_dict = TNGCluster_all_z0.tau_dict
Mstarz0 = tau_dict[Mstarz0_key]
Mstarz0.size, Mstarz0[Mstarz0 > 10.**(10.0)].size, Mstarz0[Mstarz0 > 10.**(10.5)].size, Mstarz0[Mstarz0 > 10.**(11.0)].size, Mstarz0[Mstarz0 > 10.**(11.5)].size

In [ ]:
Sim = TNGCluster_all_z0_gasz0
add_fgas_dset(Sim)
tau_dict = Sim.tau_dict
fgasz0 = tau_dict[fgas_z0_key]
mask = fgasz0 > 0
fig, ax = plt.subplots()
ax.set_yscale('log')
ax.hist(np.log10(fgasz0[mask]))

In [ ]:
fgasz0.size, fgasz0[fgasz0 < 1.0e-2].size, fgasz0[fgasz0 < 1.0e-1].size

In [ ]:
print(TNGCluster_all.tau_dict['SubfindID'].size)
print(TNGCluster_all_gasz0.tau_dict['SubfindID'].size, TNGCluster_all_nogasz0.tau_dict['SubfindID'].size)


In [ ]:
### 1D histogram plots binned by taudict_keys (jellyfish, nonjellyf, inspected) ###
def plot_tracers_mass_distributions_z0(Sims, savefig=False):
    """
    Plot the 1D histograms for 6 panels defined below. Could be generalized...
    Sims should be a list of TNGSim classes, starting with the broadest possible sample (i.e., inspected, tot)
    """

    x_keys   = [Mstarz0_key, M200cz0_key, muz0_key, MCGasz0_key, MHGasz0_key, MTGasz0_key]
    x_labels = [Mstarz0_label, M200cz0_label, muz0_label, MCGasz0_label, MHGasz0_label, MTGasz0_label]
    
    fig, axs = plt.subplots(2, 3, sharey='row')
    axs[0,0].set_ylabel(Nbranches_label, **axislabel_kwargs)
    axs[1,0].set_ylabel(Nbranches_label, **axislabel_kwargs)
    
    axs, legend_handels = plot_1Dhistograms(Sims, axs, x_keys, x_labels, y_step=0.15)
    for ax in axs.flatten():
        ax = plot_all_logticks(ax, plot_minor_ticks=True)
    
    fig.legend(handles=legend_handels, ncol=len(Sims), bbox_to_anchor=(0.5, 1.05), loc='lower center', bbox_transform=axs[0,1].transAxes,
               title=Sims[0].sim_label, title_fontsize='large', **legend_kwargs)
    
    if (savefig):
        save_label = ''
        for Sim in Sims:
            save_label += Sim.save_label
            if Sim != Sims[-1]:
                save_label += '_'
        fname = '%s_massdistributions_z0.pdf'%save_label
        for direc in direc_list:
            fig.savefig(direc + fname, bbox_inches='tight')

    return fig, axs

def plot_1Dhistograms(Sims, axs, x_keys, x_labels,
                      x_text=0.05, y_text=0.95, y_step=0.1, ha='left', va='top',
                      plot_mask=True):
    """
    fill the panels for 1D histograms and return fig, ax, and legend_handels
    Also adds the number of points with x <= 0. 
    """
    if isinstance(x_keys, six.string_types):
        x_keys = [x_keys]
    if isinstance(x_labels, six.string_types):
        x_labels = [x_labels]
    
    if len(axs.flat) != len(x_keys) or len(axs.flat) != len(x_labels):
        print('Incorrect number of axs, x_keys, or x_labels. Retunrning')
        return
    
    for ax_i, ax in enumerate(axs.flat):
        x_key   = x_keys[ax_i]
        x_label = x_labels[ax_i]
        ax.set_xlabel(x_label, **axislabel_kwargs)
        ax.set_yscale('log')

        ax = plot_1Dhist_panel(Sims, ax, x_key, plot_mask=plot_mask) 
                
    # generate legend handels 
    legend_handels = add_histlegend(Sims)
    
    return axs, legend_handels
 
def plot_1Dhist_panel(Sims, ax, x_key,
                      plot_mask=True):
    """
    Calculate the 1D histogram and add to an existing axis.
    Loop over the taudict_keys and add each to the same axis.
    For the x values <= 0, there's the option either to plot them 
    at 1.0e-2 * min(x), or to omit them entirely. 
    Returns the ax and the number of points where x <= 0
    """
    
    ### use the first Sim to set the bins ###
    x = Sims[0].tau_dict[x_key].copy()
    
    # log bins (for mass quantities) or linear bins (for time quantities)
    if (tau100_key in x_key):
        x_loval = 0
        mask = x <= x_loval
        min_plot = x.max()
        x[mask] = min_plot
        binwidth = time_binwidth
        bins, bincents = ru.returnbins(x, binwidth)
    elif (tstrip_key in x_key):
        x_loval = 0.1
        mask = x <= x_loval
        min_plot = -10.
        x[mask] = min_plot
        binwidth = time_binwidth
        bins, bincents = ru.returnbins(x, binwidth)
    elif ('T' in x_key) or ('tau' in x_key):
        x_loval = 0.1
        mask = x <= x_loval
        min_plot = 0.
        x[mask] = min_plot
        binwidth  = time_binwidth
        bins, bincents = ru.returnbins(x, binwidth)
    elif ('M' in x_key):
        x_loval = 9.0e6
        mask = x <= x_loval
        min_plot = 1.0e6
        x[mask] = min_plot
        binwidth = mass_binwidth
        bins, bincents = ru.returnlogbins(x, binwidth) 
        ax.set_xscale('log')
    elif ('mu' in x_key):
        x_loval = 1.0e-7
        mask = x <= x_loval
        min_plot = x_loval
        x[mask] = min_plot
        binwidth = mass_binwidth
        bins, bincents = ru.returnlogbins(x, binwidth) 
        ax.set_xscale('log')
        
    for Sim in Sims:
        # load dataset and validate
        x = Sim.tau_dict[x_key].copy()
        mask = x <= x_loval
        print(Sim.save_label, mask.size, mask[mask].size)

        if plot_mask:
            x[mask] = min_plot
        else:
            x = x[~mask]
            
        label = Sim.gasz0_key + ' (%d)'%len(x)
        histkwargs = Sim.histkwargs

        ax.hist(x, bins, label=label, **histkwargs)
    # finish loop over Sims
        
    return ax


def add_histlegend(Sims):
    """
    generates the legend handles for the histogram plots, including the
    total number of points contained in the histogram
    """
    result = []
    for Sim in Sims:
        histkwargs = Sim.histkwargs
        label = Sim.taudict_label + ' (%d)'%(Sim.tau_dict[M200cz0_key].size)
        result.append(Patch(facecolor='None', edgecolor=histkwargs['color'],
                            lw=histkwargs['lw'], label=label))
    return result


def plot_all_logticks(ax, x_axis=True, plot_minor_ticks=False):
    """
    helper function that ensures that each base 10 power is shown.
    assumes to be run on x-axis, but functions similarly for y-axis.
    """
    if x_axis:
        x_lims = ax.get_xlim()
        x_lolog = ru.ceil_to_value(np.log10(x_lims[0]), 1.0)
        x_hilog = ru.floor_to_value(np.log10(x_lims[1]), 1.0)
        Ndex = x_hilog - x_lolog + 1
        
        ax.xaxis.set_major_locator(mpl.ticker.LogLocator(base=10, numdecs=Ndex, numticks=Ndex))
        if plot_minor_ticks:
            locmin = mpl.ticker.LogLocator(subs=(0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9))
            ax.xaxis.set_minor_locator(locmin)
            
    else:
        ylims = ax.get_ylim()
        y_lolog = ru.ceil_to_value(np.log10(y_lims[0]), 1.0)
        y_hilog = ru.floor_to_value(np.log10(y_lims[1]), 1.0)
        Ndex = yhilog - y_lolog + 1

        ax.yaxis.set_major_locator(mpl.ticker.LogLocator(base=10, numdecs=Ndex, numticks=Ndex))
        if plot_minor_ticks:
            locmin = mpl.ticker.LogLocator(subs=(0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9))
            ax.yaxis.set_minor_locator(locmin)
            
    return ax

    

In [ ]:
savefig=False

In [ ]:
Sim = TNGCluster_all_z0
tau_dict = Sim.tau_dict

x_key = M200cz0_key
y_key = Mstarz0_key

x_label = M200cz0_label
y_label = Mstarz0_label

x = np.log10(tau_dict[x_key])
y = np.log10(tau_dict[y_key])

indices = (x > 0) & (y > 0)

binwidth = 0.05

x_bins, x_bincents = ru.returnbins(x, binwidth)
y_bins, y_bincents = ru.returnbins(y, binwidth)    

# generate 2D histogram and smooth values
hist = np.histogram2d(x[indices], y[indices], bins=[x_bins, y_bins])
vals = hist[0].T
x_hist = np.sum(vals, axis=0)
y_hist = np.sum(vals, axis=1)

fig = plt.figure(figsize=(figsizewidth * 2/3, figsizewidth * 2./3. * 2./3.))
gs = gridspec.GridSpec(2, 2, figure=fig, width_ratios=(5, 1), height_ratios=(1, 4),
                       left=0.1, right=0.9, bottom=0.1, top=0.9,
                       wspace=0.05, hspace=0.05)

ax10 = fig.add_subplot(gs[1, 0])
ax10.set_ylabel(r'$\log_{10}[$'+ y_label + r'$]$')
ax10.set_xlabel(r'$\log_{10}[$'+ x_label + r'$]$')
h = ax10.hist2d(x, y, bins=(x_bins, y_bins), cmap='Greys', norm=mpl.colors.LogNorm())
cax10 = inset_axes(ax10, width='50%', height='5%', loc='upper left')
cbar = plt.colorbar(h[3], cax=cax10, orientation='horizontal')
cbar.set_label(Nbranches_label, labelpad=5, fontsize='small')
cbar.ax.tick_params(labelsize='x-small')

ax00 = fig.add_subplot(gs[0, 0], sharex=ax10)
ax00.set_ylabel(Nbranches_label)
ax00.set_yscale('log')
ax00.step(x_bincents, x_hist, where='mid', marker='None', ls='-')
ax00.tick_params(axis="x", labelbottom=False)

ax11 = fig.add_subplot(gs[1, 1], sharey=ax10)
ax11.set_xlabel(Nbranches_label)
ax11.set_xscale('log')
ax11.step(y_hist, y_bincents, where='mid', marker='None', ls='-')
ax11.tick_params(axis="y", labelleft=False)

fig.suptitle('TNG-Cluster All (%d)'%x.size) 


In [ ]:
MGasz0_key = 'SubhaloGasMass_z0'
tau_dict = TNGCluster_all_z0.tau_dict
unique = np.unique(tau_dict['HostSubhaloGrNr'])
M200c_unique = np.zeros(unique.size)
Nsat_100 = M200c_unique.copy()
Nsat_105 = M200c_unique.copy()
Nsat_110 = M200c_unique.copy()
Nsats_Mstar = [Nsat_100,
               Nsat_105,
               Nsat_110]

vals_Mstar = [10.**(9.0),
              10.**(10.0),
              10.**(11.0)]

Nsat_070 = M200c_unique.copy()
Nsat_090 = M200c_unique.copy()
Nsat_100 = M200c_unique.copy()
Nsats_gas = [Nsat_070,
             Nsat_090,
             Nsat_100]
vals_gas = [10.**(7.0),
            10.**(9.0),
            10.**(10.0)]


for halo_i, haloID in enumerate(unique):
    sat_indices = tau_dict['HostSubhaloGrNr'] == haloID
    Mstar_sats = tau_dict[Mstarz0_key][sat_indices]
    Mgas_sats = tau_dict[MGasz0_key][sat_indices]
    M200c_unique[halo_i] = tau_dict[M200cz0_key][sat_indices][0]
    for val_i, val in enumerate(vals_Mstar):
        Nsats_Mstar[val_i][halo_i] = np.where(Mstar_sats > vals_Mstar[val_i])[0].size
        Nsats_gas[val_i][halo_i] = np.where(Mgas_sats > vals_gas[val_i])[0].size

fig, axs = plt.subplots(2, 1, figsize=(figsizewidth_column, figsizewidth_column*1.25))
colors_Mstar = ['darkgoldenrod', 'orange', 'bisque']
colors_Mstar = ['bisque', 'orange', 'darkgoldenrod']

colors_gas = ['violet', 'mediumpurple', 'indigo']


Nsats_list = [Nsats_Mstar, Nsats_gas]
vals_list = [vals_Mstar, vals_gas]
colors_list = [colors_Mstar, colors_gas]

binwidth_log = 0.1

for ax_i, ax in enumerate(axs): 
    ax.set_xscale('log')
    ax.set_xlabel(M200cz0_label)
    ax.set_ylabel(r'\# Satellites per Cluster')
    ax.set_yscale('log')
    
    Nsats = Nsats_list[ax_i]
    vals = vals_list[ax_i]
    colors = colors_list[ax_i]
    for val_i, val in enumerate(vals):
        Nsat = Nsats[val_i]
        color = colors[val_i]
        bin_cents, y_50s, y_16s, y_84s = ru.return2dhiststats(np.log10(M200c_unique), Nsat, binwidth_log)
        
        #ax.plot(M200c_unique, Nsat, c=color, fillstyle='none')
        ax.plot(10.**(bin_cents), y_50s + 1, c=color, ls='-', lw=4, marker='None', label=r'$10^{%.1f}$'%np.log10(val))
        ax.fill_between(10.**(bin_cents), y_16s + 1, y_84s + 1, color=color, alpha=0.2)
        
    if ax_i == 0:
        ax.legend(title=r'$M_\star^{\rm sat} / {\rm M_\odot} > $', fontsize='small', ncol=2, loc='upper left')
        ax.set_title(r'TNG-Cluster Halo Richness')
        ax.text(0.95, 0.05, r'$z=0$ (352 halos)', ha='right', va='bottom', transform=ax.transAxes)
    else:
        ax.legend(title=r'$M_{\rm Gas}^{\rm sat} / {\rm M_\odot} > $', fontsize='small', ncol=2, loc='upper left')
 

In [ ]:
fig, ax = plt.subplots(figsize=(figsizewidth_column, figsizeheight_column*1.25))

colors_gas = ['violet', 'mediumpurple', 'indigo']

Nsats_list = [Nsats_Mstar, Nsats_gas]
vals_list = [vals_Mstar, vals_gas]
colors_list = [colors_Mstar, colors_gas]

binwidth_log = 0.1

ax.set_xscale('log')
ax.set_xlabel(M200cz0_label)
ax.set_ylabel(r'\# Satellites per Cluster')
ax.set_yscale('log')
ax_i = 1
Nsats = Nsats_list[ax_i]
vals = vals_list[ax_i]
colors = colors_list[ax_i]

ax.set_title('TNG-Cluster FoF Satellites \n '+ r'$M_{\star}^{\rm sat} > 10^9\, {\rm M_\odot}$ (%d)'%(Nsats_Mstar[0].sum()))
ax.text(0.95, 0.05, r'$z=0$ (352 halos)', ha='right', va='bottom', transform=ax.transAxes)

for val_i, val in enumerate(vals):
    Nsat = Nsats[val_i]
    color = colors[val_i]
    bin_cents, y_50s, y_16s, y_84s = ru.return2dhiststats(np.log10(M200c_unique), Nsat, binwidth_log)
    
    #ax.plot(M200c_unique, Nsat, c=color, fillstyle='none')
    ax.plot(10.**(bin_cents), y_50s + 1, c=color, ls='-', lw=4, marker='None', label=r'$10^{%.1f}$'%np.log10(val))
    ax.fill_between(10.**(bin_cents), y_16s + 1, y_84s + 1, color=color, alpha=0.2)
    
    ax.legend(title=r'$M_{\rm Gas}^{\rm sat} / {\rm M_\odot} > $', fontsize='small', ncol=2, loc='upper left')

if savefig:
    fname = 'L680n8192TNG_halorichness-Mgas.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')
 

In [ ]:
# load the primary zoom target centrals and all subhalos in the simulation
indirec = '../Output/L680n8192TNG_subfindGRP/'
infname = 'central_subfind_L680n8192TNG_tau_all.hdf5'
centrals_taudict = {}
with h5py.File(indirec + infname, 'r') as f:
    group = f['Group']
    for dset_key in group.keys():
        centrals_taudict[dset_key] = group[dset_key][:]
    f.close()
    
haloIDs = centrals_taudict['HostSubhaloGrNr']
M200cz0 = centrals_taudict[M200cz0_key]

sim = 'L680n8192TNG'
basePath = ru.ret_basePath(sim)
snapNum = 99

# load general simulation parameters
header = il.groupcat.loadHeader(basePath, snapNum)
z = header['Redshift']
a = header['Time'] # scale factor
h = header['HubbleParam'] # = 0.6774
boxsize = header['BoxSize'] * a / h
    
gas_ptn = il.util.partTypeNum('gas')

# load all subhalos in the simulation
subhalo_fields = ['SubhaloMassInRadType', 'SubhaloPos', 'SubhaloMassType', 'SubhaloStellarPhotometrics']
star_ptn = il.util.partTypeNum('star')

Subhalos = il.groupcat.loadSubhalos(basePath, snapNum, fields=subhalo_fields)
Subhalos_Mstar = Subhalos['SubhaloMassInRadType'][:,star_ptn] * 1.0e10 / h
Subhalos_Pos = Subhalos['SubhaloPos'] * a / h
Subhalos_Mgas = Subhalos['SubhaloMassType'][:,gas_ptn] * 1.0e10 / h
Subhalos_Vmag = Subhalos['SubhaloStellarPhotometrics'][:,2]
Subhalos_Rmag = Subhalos['SubhaloStellarPhotometrics'][:,5]

proj = 'xy'
axes = [0,1]

Mstar_lolim = 1.0e10 # Msun
Mstar_hilim_frac = 1. / 10. # M_star^cent
Vmag_hilim = -19
Rmag_hilim = -20.44

FoV_fact_rvir = 1.0 # Rvir

Ngal_vmag = np.zeros(len(haloIDs))
Ngal_rmag = Ngal_vmag.copy()
for i, haloID in enumerate(haloIDs):
    halo = il.groupcat.loadSingle(basePath, snapNum, haloID=haloID)
    M200c = halo['Group_M_Crit200'] * 1.0e10 / h
    R200c = halo['Group_R_Crit200'] * a / h
    halo_pos = halo['GroupPos'] * a / h
    central = il.groupcat.loadSingle(basePath, snapNum, subhaloID=halo['GroupFirstSub'])
    central_Mstar = central['SubhaloMassInRadType'][star_ptn] * 1.0e10 / h   

    subhalo_indices = ((Subhalos_Vmag <= Vmag_hilim) & 
                       (np.sqrt((Subhalos_Pos[:,axes[0]] - halo_pos[axes[0]])**2 +
                                (Subhalos_Pos[:,axes[1]] - halo_pos[axes[1]])**2) < FoV_fact_rvir * R200c))
    
    subhaloIDs = np.where(subhalo_indices)[0]

    Ngal_vmag[i] = subhaloIDs.size

    subhalo_indices = ((Subhalos_Rmag <= Rmag_hilim) & 
                       (np.sqrt((Subhalos_Pos[:,axes[0]] - halo_pos[axes[0]])**2 +
                                (Subhalos_Pos[:,axes[1]] - halo_pos[axes[1]])**2) < FoV_fact_rvir * R200c))
    
    subhaloIDs = np.where(subhalo_indices)[0]

    Ngal_rmag[i] = subhaloIDs.size
 

fig, ax = plt.subplots(figsize=(figsizewidth_column, figsizeheight_column))
ax.set_xscale('log')
ax.set_xlabel(M200cz0_label)
ax.set_ylabel(r'\# Satellites per Cluster')
ax.set_yscale('log')
ax.plot(M200cz0, Ngal_vmag, color='black', ms=2, marker='o', mew=1, label=r'$r_{\rm mag} < -20.4$', fillstyle='none')
ax.legend(title=r'proj. $d_{\rm sat}^{\rm host} < R_{\rm 200c}^{\rm host}$')
if savefig:
    fname = 'L680n8192TNG_proj-halorichness-Vmag.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')

In [ ]:
# data from Abdullah+23, taken using WebPlotDigitizer
N200_mean = 10.**(np.array([1.107142857, 1.358695652, 1.599378882, 1.857142857, 2.104037267]))
N200_lolim = 10.**(np.array([1.032608696, 1.288819876, 1.526397516, 1.787267081, 2.057453416]))
N200_hilim = 10.**(np.array([1.184782609, 1.430124224, 1.673913043, 1.925465839, 2.149068323]))
M200c_mean = 10.**(np.array([14.12939698, 14.19623116, 14.3879397, 14.70452261, 14.92964824])) / 0.6774
M200c_lolim = 10.**(np.array([13.96758794, 14.02386935, 14.24020101, 14.57437186, 14.80829146])) / 0.6774
M200c_hilim = 10.**(np.array([14.29120603, 14.37035176, 14.53919598, 14.83643216, 15.04748744])) / 0.6774

# data from Abdullah+23, taken using WebPlotDigitizer
N200_mean = 10.**(np.array([1.358695652, 1.599378882, 1.857142857, 2.104037267]))
N200_lolim = 10.**(np.array([1.288819876, 1.526397516, 1.787267081, 2.057453416]))
N200_hilim = 10.**(np.array([1.430124224, 1.673913043, 1.925465839, 2.149068323]))
M200c_mean = 10.**(np.array([14.19623116, 14.3879397, 14.70452261, 14.92964824])) / 0.6774
M200c_lolim = 10.**(np.array([14.02386935, 14.24020101, 14.57437186, 14.80829146])) / 0.6774
M200c_hilim = 10.**(np.array([14.37035176, 14.53919598, 14.83643216, 15.04748744])) / 0.6774

In [ ]:
# load all halos and find the primary zoom target IDs
halo_fields = ['GroupPrimaryZoomTarget', 'Group_M_Crit200', 'Group_M_Mean200']
halos = il.groupcat.loadHalos(basePath, 99, fields=halo_fields)
haloIDs = np.where(halos['GroupPrimaryZoomTarget'] == 1)[0]
M200m_M200c_ratio = np.median(halos['Group_M_Mean200'][haloIDs] / halos['Group_M_Crit200'][haloIDs])
  

In [ ]:
# data from Costanzi+19, taken from Table 1
lambda_mean = np.array([23.62, 32.39, 43.49, 59.04, 98.50]) / M200m_M200c_ratio
lambda_lo = np.array([20, 27.9, 37.6, 50.3, 69.3]) / M200m_M200c_ratio
lambda_hi = np.array([27.9, 37.6, 50.3, 69.3, 140]) / M200m_M200c_ratio
M200m_mean = 10.**(np.array([14.111, 14.263, 14.38, 14.609, 14.928])) / 0.6774 / M200m_M200c_ratio
M200m_err = M200m_mean * (1 - 10.**(np.array([0.035, 0.038, 0.042, 0.046, 0.046])))

lambda_mean = np.array([32.39, 43.49, 59.04, 98.50]) / M200m_M200c_ratio
lambda_lo = np.array([27.9, 37.6, 50.3, 69.3]) / M200m_M200c_ratio
lambda_hi = np.array([37.6, 50.3, 69.3, 140]) / M200m_M200c_ratio
M200m_mean = 10.**(np.array([14.263, 14.38, 14.609, 14.928])) / 0.6774 / M200m_M200c_ratio
M200m_err = M200m_mean * (1 - 10.**(np.array([0.038, 0.042, 0.046, 0.046])))

In [ ]:
fig, ax = plt.subplots(figsize=(figsizewidth_column, figsizeheight_column))
ax.set_xscale('log')
ax.set_xlabel(M200cz0_label)
ax.set_ylabel(r'\# Satellites per Cluster')
ax.set_yscale('log')
ax.plot((M200cz0), (Ngal_rmag), color='black', ms=2, marker='o', mew=1, label=r'$r_{\rm mag} < -20.44$', fillstyle='none', zorder=1)

ax.errorbar(M200c_mean, N200_mean, xerr=[M200c_mean-M200c_lolim, M200c_hilim-M200c_mean], yerr=[N200_mean-N200_lolim, N200_hilim-N200_mean], color='tab:orange', marker='None')
ax.errorbar(M200m_mean, lambda_mean, xerr=M200m_err, yerr=[lambda_mean-lambda_lo, lambda_hi-lambda_mean], color='tab:green', marker='None')

ax.legend(title=r'proj. $d_{\rm sat}^{\rm host} < R_{\rm 200c}^{\rm host}$')
if savefig:
    fname = 'L680n8192TNG_proj-halorichness-Vmag.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')

In [ ]:


subhalo = il.groupcat.loadSubhalos(basePath, 99, fields=['SubhaloMassType'])
subhalo_totmasses = np.sum(subhalo, axis=1) * 1.0e10 / h
subhalo_lowresmasses = subhalo[:2] * 1.0e10 / h


In [ ]:
M200cz0_label = r'Host Cluster Mass [$M_{\rm 200c}^{\rm host} / {\rm M_\odot}$]'

fig = plt.figure(figsize=(figsizewidth, figsizewidth / 1.2))
gs0 = gridspec.GridSpec(2, 12, figure=fig, height_ratios=(2,1))

gs_top = gridspec.GridSpecFromSubplotSpec(2, 2, subplot_spec=gs0[0,1:-1],
                                          width_ratios=(5, 1), height_ratios=(1, 4),
                                          wspace=0.05, hspace=0.05)

ax10 = fig.add_subplot(gs_top[1, 0])
ax10.set_ylabel(r'Satellite Stellar Mass [$\log_{10} M_{\star}^{\rm sat} / {\rm M_\odot}$]')
ax10.set_xlabel(r'Host Cluster Mass [$\log_{10} M_{\rm 200c}^{\rm host} / {\rm M_\odot}$]')
h = ax10.hist2d(x, y, bins=(x_bins, y_bins), cmap='Greys', norm=mpl.colors.LogNorm())
cax10 = inset_axes(ax10, width='50%', height='5%', loc='upper left')
cbar = plt.colorbar(h[3], cax=cax10, orientation='horizontal')
cbar.set_label(Nbranches_label, labelpad=5, fontsize='small')
cbar.ax.tick_params(labelsize='x-small')

ax10.axhline(10.0, color='tab:orange', lw=4, ls='-', marker='None')

trans = transforms.blended_transform_factory(ax10.transAxes, ax10.transData)

annotate_kwargs = dict(xytext=((0.05),(10.0)), xycoords=trans,
                           arrowprops=dict(facecolor='tab:orange', shrink=0.05),
                           fontsize='medium', ha='right', va='center', c='tab:orange')
ax10.annotate('', (0.05, 10.75), **annotate_kwargs)
ax10.text(0.09, 10.1, 'Massive \n Satellites', transform=trans, ha='left', va='bottom', c='blanchedalmond', fontsize='x-large', fontweight='black')


ax00 = fig.add_subplot(gs_top[0, 0], sharex=ax10)
ax00.set_ylabel(Nbranches_label)
ax00.set_yscale('log')
ax00.step(x_bincents, x_hist, where='mid', marker='None', ls='-')
ax00.tick_params(axis="x", labelbottom=False)

ax11 = fig.add_subplot(gs_top[1, 1], sharey=ax10)
ax11.set_xlabel(Nbranches_label)
ax11.set_xscale('log')
ax11.step(y_hist, y_bincents, where='mid', marker='None', ls='-')
ax11.tick_params(axis="y", labelleft=False)

gs_bot = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs0[1,:])

ax0 = fig.add_subplot(gs_bot[0])

ax0.set_xscale('log')
ax0.set_xlabel(M200cz0_label)
ax0.set_ylabel(r'\# Satellites per Cluster')
ax0.set_yscale('log')

Nsats = Nsats_list[0]
vals = vals_list[0]
colors = colors_list[0]
for val_i, val in enumerate(vals):
    Nsat = Nsats[val_i]
    color = colors[val_i]
    bin_cents, y_50s, y_16s, y_84s = ru.return2dhiststats(np.log10(M200c_unique), Nsat, binwidth_log)
    
    ax0.plot(10.**(bin_cents), y_50s, c=color, ls='-', lw=4, marker='None', label=r'$10^{%.1f}$'%np.log10(val))
    ax0.fill_between(10.**(bin_cents), y_16s, y_84s, color=color, alpha=0.2)
    
ax0.legend(title=r'$M_\star^{\rm sat} / {\rm M_\odot} > $', fontsize='small', ncol=2, loc='upper left')
#ax0.set_title(r'TNG-Cluster Halo Richness')
ax0.text(0.99, 0.05, r'only FoF satellites.', ha='right', va='bottom', transform=ax0.transAxes)
ax0.set_ylim(1.0e0, 2.0e3)

ax1 = fig.add_subplot(gs_bot[1], sharey=ax0)

ax1.set_xscale('log')
ax1.set_xlabel(M200cz0_label)
ax1.set_ylabel(r'\# Satellites per Cluster')
ax1.set_yscale('log')
ax1.plot(M200cz0, Ngal_rmag, color='tab:gray', ms=2, marker='o', mew=1, label=r'TNG-Cluster ($z=0)$', fillstyle='none', zorder=1)

data_kwargs = dict(elinewidth=2, capsize=2.5, capthick=2, marker='None')
ax1.errorbar(M200c_mean, N200_mean, xerr=[M200c_mean-M200c_lolim, M200c_hilim-M200c_mean], yerr=[N200_mean-N200_lolim, N200_hilim-N200_mean], color='tab:orange', label=r'GalWCat19 ($z\approx0.1$)', **data_kwargs)
ax1.errorbar(M200m_mean, lambda_mean, xerr=M200m_err, yerr=[lambda_mean-lambda_lo, lambda_hi-lambda_mean], color='tab:green', label=r'redMaPPer ($z=0.1-0.3$)', **data_kwargs)
ax1.legend(loc='upper left', ncol=1, fontsize='small')
text = ('all galaxies (%d) \n '%(Ngal_rmag.sum()) +
        r'brighter than $r_{\rm mag} < -20.4$' + '\n and within ' + 
        r'proj. $d_{\rm sat}^{\rm host} < R_{\rm 200c}^{\rm host}$'
        )
  
ax1.text(0.975, 0.05, text, ha='right', va='bottom', ma='right', transform=ax1.transAxes)

fig.suptitle(r'TNG-Cluster: %d halos, %d FoF satellites with $M_{\star}^{\rm sat} > 10^9\, {\rm M_\odot}$ at $z=0$'%(haloIDs.size, x.size))

if savefig:
    fname = 'L680n8192TNG_halorichness_multipanel.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')


In [ ]:
# plot Mstar vs M_V to get an idea for which magnitude M* >~ 10^10 Msun is
mag_key = 'SubhaloStellarPhotometricsr_z0'
x = np.log10(tau_dict[Mstarz0_key])
y = tau_dict[mag_key]

binwidth = 0.05

fig, ax = plt.subplots()

sigmas = [2, 1]
percentiles = [2.5, 16, 50, 84, 97.5]

stats = ru.return2dhiststats_dict(x, y, binwidth, percentiles)
bin_cents = 10**(stats['bin_cents'])

for i, sigma in enumerate(sigmas):
    lo_percentile = percentiles[i]
    hi_percentile = percentiles[-(i+1)]
    
    y_los = stats[lo_percentile]
    y_his = stats[hi_percentile]
    
    ax.fill_between(bin_cents, y_los, y_his, color='tab:gray', alpha=1./(sigma+3.))

y_50s = stats[50]
ax.plot(bin_cents, y_50s, c='dimgray', ls='-', lw=4, marker='None')

ax.set_xscale('log')
ax.set_title('TNG-Cluster All (%d)'%x.size) 
ax.set_xlabel(Mstarz0_label)
ax.set_ylabel(r'$r$ band magnitude')
ax.axvline(10.**(10.4), marker='None', ls='-')
ax.axhline(-20.44, marker='None', ls='-')
#ax.vlines(1.0e10, 0.0, 1.0, transform=ax.get_xaxis_transform())
fname = '%s_rmag-mstarz0.pdf'%Sim.save_label

if (savefig):
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')

In [ ]:
def plot_2dhist_smoothed(Sim, ax, x_key, y_key,
                         loc='lower right'):
    """
    fill an ax with a gaussian smoothed 2d histogram of a given taudict_key (jellyfish, nonjellyf, inspected).
    requires the x the y keys for Sim.tau_dict[taudict_key], and optionally the location of the colorbar.
    Assumes equal bins for the x and y data, and plots the one to one line. used for comparing two definitions.
    returns the ax.
    """
    
    tau_dict = Sim.tau_dict
    
    x = tau_dict[x_key]
    y = tau_dict[y_key]
    
    indices = (x > 0) & (y > 0)
    
    # log bins (for mass quantities) or linear bins (for time quantities)
    if 'Tstrip' in x_key:
        binwidth = time_binwidth
        bins, bincents = ru.returnbins([0.01, 9.99], binwidth)
    elif 'T' in x_key:
        binwidth  = time_binwidth
        bins, bincents = ru.returnbins([0.01, 13.9], binwidth)
    elif ('M' in x_key) or ('mu' in x_key):
        binwidth = mass_binwidth
        ax.set_xscale('log')
        bins, bincents = ru.returnlogbins(y, binwidth)      
        
    # generate 2D histogram and smooth values
    hist = np.histogram2d(y[indices], x[indices], bins=bins)
    #vals = gaussian_filter(hist[0], binwidth)
    vals = hist[0]
    
    # plot contours based on percentiles of the smoothed values
    percentiles = [50, 75, 95]
    levels = np.zeros(len(percentiles))
    for i, p in enumerate(percentiles):
        levels[i] = np.percentile(vals[vals > 0], p)
        
    # plot the filled contours
    ax.contour(bincents, bincents, vals, levels=levels, cmap=Sim.cmap, extend='both')
    cs = ax.contourf(bincents, bincents, vals, levels=levels, cmap=Sim.cmap, extend='both')
    
    # add the colorbar 
    cax = inset_axes(ax, width='50%', height='5%', loc=loc) 
    cbar = plt.colorbar(cs, cax=cax, orientation='horizontal')
    cbar.set_label(r'Percentiles', labelpad=5, fontsize='small')
    cax.set_xticklabels(['%d'%(p) for p in percentiles])
    cbar.ax.tick_params(labelsize='x-small')
    cax.minorticks_off()
    if loc == 'lower right':
        cax.xaxis.set_label_position('top')
        cax.xaxis.set_ticks_position('top')
                
    return ax


In [ ]:
def split_Sim_highlowmass(sim, taudict_key, gasz0_key, Mstarz0_limit=10.**(11)):
    """
    Create two instances of the TNGSim, but split the tau_dict into high and low
    mass based on the Mstarz0_limit. Also update the respective labels.
    """
    LowMass = TNGSim(sim, taudict_key, gasz0_key)
    HighMass = TNGSim(sim, taudict_key, gasz0_key)
        
    tau_dict = LowMass.tau_dict
    lowmass_indices = tau_dict[Mstarz0_key] <= Mstarz0_limit
    for key in tau_dict.keys():
        tau_dict[key] = tau_dict[key][lowmass_indices]
        HighMass.tau_dict[key] = HighMass.tau_dict[key][~lowmass_indices]
        
    if Mstarz0_limit == 10.**(11):
        Mstarz0_limit_label = '11'
    elif Mstarz0_limit == 10.**(10.5):
        Mstarz0_limit_label = '10.5'
    else: 
        Mstarz0_limit_label = '%.2f'%(np.log10(Mstarz0_limit))
    LowMass.plot_label = ('%s %s \n %s \n'%(LowMass.sim_label, LowMass.taudict_label, LowMass.gasz0_label) + 
                          r'$M_{\star}^{\rm sat} = 10^{10-%s}\, {\rm M_\odot}$ (%d)'%(Mstarz0_limit_label, lowmass_indices[lowmass_indices].size))
    LowMass.save_label = sim + '_' + taudict_key + '_' + gasz0_key + '_lowmass'
    
    HighMass.plot_label = ('%s %s \n %s \n'%(HighMass.sim_label, HighMass.taudict_label, HighMass.gasz0_label) + 
                          r'$M_{\star}^{\rm sat} = 10^{%s-12.5}\, {\rm M_\odot}$ (%d)'%(Mstarz0_limit_label, lowmass_indices[~lowmass_indices].size))
    HighMass.save_label = sim + '_' + taudict_key + '_' + gasz0_key + '_highmass'
    
    return LowMass, HighMass
    
    

In [ ]:
TNGCluster_all_tot_lowmass, TNGCluster_all_tot_highmass = split_Sim_highlowmass(sim, all_key, tot_key)
TNGCluster_all_nogas_lowmass, TNGCluster_all_nogas_highmass = split_Sim_highlowmass(sim, all_key, nogas_key)

TNGCluster_clean_tot_lowmass, TNGCluster_clean_tot_highmass = split_Sim_highlowmass(sim, clean_key, tot_key)
TNGCluster_clean_nogas_lowmass, TNGCluster_clean_nogas_highmass = split_Sim_highlowmass(sim, clean_key, nogas_key)

TNGCluster_PP_tot_lowmass, TNGCluster_PP_tot_highmass = split_Sim_highlowmass(sim, PP_key, tot_key)
TNGCluster_PP_nogas_lowmass, TNGCluster_PP_nogas_highmass = split_Sim_highlowmass(sim, PP_key, nogas_key)

TNGCluster_BP_tot_lowmass, TNGCluster_BP_tot_highmass = split_Sim_highlowmass(sim, BP_key, tot_key)
TNGCluster_BP_nogas_lowmass, TNGCluster_BP_nogas_highmass = split_Sim_highlowmass(sim, BP_key, nogas_key)

In [ ]:
tau0_key = 'CosmicTime_tau_infall_HotGas0'
tau0_label = r'$\tau_{\rm FirstInfall}$ [Gyr]'

tau0_medpeak_Mass_key = 'CosmicTime_tau_medpeak_Mass0'
tau0_medpeak_HotGas_key = 'CosmicTime_tau_medpeak_HotGas0'
tau0_medpeak_ColdGas_key = 'CosmicTime_tau_medpeak_ColdGas0'
tau100_medpeak_HotGas_key = 'CosmicTime_tau_medpeak_HotGas100'
tau100_medpeak_ColdGas_key = 'CosmicTime_tau_medpeak_ColdGas100'

tau0_medpeak_Mass_label = r'$\tau_0^{\rm Mass}$ [Gyr]'
tau0_medpeak_HotGas_label = r'$\tau_0^{\rm HotGas}$ [Gyr]'
tau0_medpeak_ColdGas_label = r'$\tau_0^{\rm ColdGas}$ [Gyr]'
tau100_medpeak_HotGas_label = r'$\tau_{100}^{\rm HotGas}$ [Gyr]'
tau100_medpeak_ColdGas_label = r'$\tau_{100}^{\rm ColdGas}$ [Gyr]'

taugasloss_medpeak_HotGas_key = 'Tstrip_tau_medpeak_HotGas_tau0-tau100'
taugasloss_medpeak_ColdGas_key = 'Tstrip_tau_medpeak_ColdGas_tau0-tau100'
taugasloss_medpeak_HotGas_label = r'$\tau_{100}^{\rm HotGas} - \tau_{0}^{\rm HotGas}$ [Gyr]'
taugasloss_medpeak_ColdGas_label = r'$\tau_{100}^{\rm ColdGas} - \tau_{0}^{\rm ColdGas}$ [Gyr]'    

def plot_tautimes_contours_comparison(Sims,
                                      x_keys=[tau0_key, tau0_key, tau0_key],
                                      x_labels=[tau0_label, tau0_label, tau0_label],
                                      y_keys=[tau0_medpeak_Mass_key, tau0_medpeak_HotGas_key, tau0_medpeak_ColdGas_key],
                                      y_labels=[tau0_medpeak_Mass_label, tau0_medpeak_HotGas_label, tau0_medpeak_ColdGas_label],
                                      savefig=False):
    """
    compare datasets for each Sim as contours of 2D hist.
    default is for 3 sims and 3 x/y dsets, whereby actually 
    the 3 x dsets are identical. 
    Note that savefig only works if 
    """
        
    if (len(y_keys) != len(y_labels)) or (len(x_keys) != len(x_labels)) or (len(x_keys) != len(y_keys)):
        raise ValueError('Improper number of x_keys, x_labels, y_keys, or y_labels given. Try again.')
        
    if not isinstance(Sims, list):
        Sims = [Sims]
    
    fig, axs = plt.subplots(len(y_keys), len(Sims), figsize=(figsizewidth*len(Sims)/3., figsizewidth*len(x_keys)/3.))
    
    for Sim_i, Sim in enumerate(Sims):
        for y_i, y_key in enumerate(y_keys):
            if len(Sims) == 1 and len(y_keys) == 1:
                ax = axs
            elif len(Sims) == 1:
                ax = axs[y_i]
            elif len(y_keys) == 1:
                ax = axs[Sim_i]
            else:
                ax = axs[y_i, Sim_i]
                        
            ax = plot_2dhist_smoothed(Sim, ax, x_keys[y_i], y_key,
                                        loc='upper left')
            ax.set_ylabel(y_labels[y_i])
            ax.set_xlabel(x_labels[y_i])
            ax.plot(ax.get_xlim(), ax.get_ylim(), **one_one_kwargs)
            ax.set_box_aspect(1)
            if y_i == 0:
                ax.set_title('%s '%(Sim.plot_label), fontsize='medium')
    
    if (savefig):
        Sims_label = ''
        for Sim in Sims:
            Sims_label += '%s_'%Sim.save_label
        dsets_label = ''
        for ykey_i, ykey in enumerate(y_keys):
            dsets_label += y_key + '-' + x_keys[ykey_i] + '_'
        fname = '%s%scontours_comparison.pdf'%(Sims_label, dsets_label)
        for direc in direc_list:
            try: 
                fig.savefig(direc + fname, bbox_inches='tight')
            except OSError as exc:
                if exc.errno == 36:
                    print('Error, filename too long. Not saving.')
                    return fig, axs
                else:
                    raise
    return fig, axs


def plot_infalltimes_contours_comparison(Sims,
                                         x_keys=[tau0_key, tau0_key, tau0_key],
                                         x_labels=[tau0_label, tau0_label, tau0_label],
                                         y_keys=[tau0_medpeak_Mass_key, tau0_medpeak_HotGas_key, tau0_medpeak_ColdGas_key],
                                         y_labels=[tau0_medpeak_Mass_label, tau0_medpeak_HotGas_label, tau0_medpeak_ColdGas_label],
                                         savefig=False):
    """
    Wrapper function to plot and save the infall times comparison
    """
    fig, axs = plot_tautimes_contours_comparison(Sims, x_keys, x_labels, y_keys, y_labels, savefig=False)
    
    if (savefig):
        Sims_label = ''
        for Sim in Sims:
            Sims_label += '%s_'%Sim.save_label
        dsets_label = 'infalltimes_'
        fname = '%s%scontours_comparison.pdf'%(Sims_label, dsets_label)
        for direc in direc_list:
            try: 
                fig.savefig(direc + fname, bbox_inches='tight')
            except OSError as exc:
                if exc.errno == 36:
                    print('Error, filename too long. Not saving.')
                    return fig, axs
                else:
                    raise
    return fig, axs


def plot_taugasloss_contours_comparison(Sims,
                                        x_keys=[tau0_medpeak_HotGas_key, tau100_medpeak_HotGas_key, taugasloss_medpeak_HotGas_key],
                                        x_labels=[tau0_medpeak_HotGas_label, tau100_medpeak_HotGas_label, taugasloss_medpeak_HotGas_label],
                                        y_keys=[tau0_medpeak_ColdGas_key, tau100_medpeak_ColdGas_key, taugasloss_medpeak_ColdGas_key],
                                        y_labels=[tau0_medpeak_ColdGas_label, tau100_medpeak_ColdGas_label, taugasloss_medpeak_ColdGas_label],
                                        savefig=False):
    """
    Wrapper function to plot and save the tau_gasloss comparisons
    """
    fig, axs = plot_tautimes_contours_comparison(Sims, x_keys, x_labels, y_keys, y_labels, savefig=False)
    
    if (savefig):
        Sims_label = ''
        for Sim in Sims:
            Sims_label += '%s_'%Sim.save_label
        dsets_label = 'taugasloss_'
        fname = '%s%scontours_comparison.pdf'%(Sims_label, dsets_label)
        for direc in direc_list:
            try: 
                fig.savefig(direc + fname, bbox_inches='tight')
            except OSError as exc:
                if exc.errno == 36:
                    print('Error, filename too long. Not saving.')
                    return fig, axs
                else:
                    raise
    return fig, axs


In [ ]:
Sims = [TNGCluster_all, TNGCluster_clean, TNGCluster_PP, TNGCluster_BP]
fig, axs = plot_infalltimes_contours_comparison(Sims, savefig=savefig)

In [ ]:

Sims = [TNGCluster_all_nogasz0, TNGCluster_clean_nogasz0, TNGCluster_PP_nogasz0, TNGCluster_BP_nogasz0]
fig, axs = plot_taugasloss_contours_comparison(Sims, savefig=savefig)

Sims = [TNGCluster_all_nogasz0, TNGCluster_all_nogas_lowmass, TNGCluster_all_nogas_highmass]
fig, axs = plot_taugasloss_contours_comparison(Sims, savefig=savefig)

In [ ]:
def plot_time_distributions(Sims,
                            x_keys=[tau0_key, tau100_key, tstrip_key],
                            x_labels=[tau0_label, tau100_label, tstrip_label],
                            savefig=False, legend=True, add_mstar_m200c_info=False):
    """
    Plot 1d histogram distributions for the given x quantities
    Default is tau0, tau100, tauRPS for the standard definition.
    """
    
    if len(x_keys) != len(x_labels):
        print('Wrong number of x_keys or x_labels.')
        return
    
    fig, axs = plt.subplots(1, len(x_keys), sharey=True, figsize=(figsizewidth, figsizewidth/2.5))
    axs[0].set_ylabel(Nbranches_label, **axislabel_kwargs)
        
    if 'Tstrip' in x_keys[0]:
        x_text = 0.95
        ha = 'right'
        save_key = 'tstrip'
    elif 'T' in x_keys[0]:
        x_text = 0.05
        ha = 'left'
        save_key = 'taus'
        
    axs, _ = plot_1Dhistograms(Sims, axs, x_keys, x_labels,
                               y_step=0.10, x_text=x_text, ha=ha, plot_mask=True)
        
    axs[0] = add_redshift_sincez2(axs[0])
    axs[1] = add_redshift_sincez2(axs[1])
    axs[2].set_xlim(-0.5, 10.1)
    
    ### add some extra info and text
    # axs[0]: clarify tau0 == infall time
    text = r'$\tau_0 \equiv$ infall time'
    textkwargs = dict(ha='left', va='top', transform=axs[0].transAxes, **text_kwargs)
    text = axs[0].text(0.05, 0.95, text, **textkwargs)
        
    # axs[1]: state the Mstar and M200c ranges for each sample
    if add_mstar_m200c_info:
        textkwargs = dict(ha='left', va='top', ma='left', transform=axs[1].transAxes, **text_kwargs)
        xval = 0.05
        yval = 0.95
        ystep = 0.11
        ymstar = yval - ystep
        ym200c = ymstar - ystep * 3
        for i, Sim in enumerate(Sims):
            if i == 0:
                axs[1].text(xval, yval, r'at $z=0:$', **textkwargs)
                axs[1].text(xval, ymstar, r'$\lg[M_{\star}^{\rm sat}] \in$', **textkwargs)
                axs[1].text(xval, ym200c, r'$\lg[M_{\rm 200c}^{\rm host}] \in$', **textkwargs)
            c = Sim.plotkwargs['color']
            mstarlolim = ru.floor_to_value(np.log10(Sim.tau_dict[Mstarz0_key].min()), 0.1)
            mstarhilim = ru.ceil_to_value(np.log10(Sim.tau_dict[Mstarz0_key].max()), 0.1)
            m200clolim = ru.floor_to_value(np.log10(Sim.tau_dict[M200cz0_key].min()), 0.1)
            m200chilim = ru.ceil_to_value(np.log10(Sim.tau_dict[M200cz0_key].max()), 0.1)
            axs[1].text(xval, ymstar - ystep*(i + 1), r'$(%.1f, %.1f)$'%(mstarlolim, mstarhilim), c=c, **textkwargs)
            axs[1].text(xval, ym200c - ystep*(i + 1), r'$(%.1f, %.1f)$'%(m200clolim, m200chilim), c=c, **textkwargs)
        
    if legend:
        legend_handels = add_histlegend_gasz0(Sims)
        fig.legend(handles=legend_handels, ncol=2, bbox_to_anchor=(0.5, 1.15), loc='lower center', bbox_transform=axs[1].transAxes,
                   title=Sims[0].sim_label + ' ' + Sims[0].taudict_label, title_fontsize='large', **legend_kwargs)
    
    if (savefig):
        save_label = ''
        for Sim in Sims:
            save_label += Sim.save_label
            if Sim != Sims[-1]:
                save_label += '_'
        fname = '%s_tau_distributions.pdf'%save_label
        for direc in direc_list:
            fig.savefig(direc + fname, bbox_inches='tight')
            
    return fig, axs

def add_histlegend_gasz0(Sims):
    """
    generates the legend handles for the histogram plots, including the
    total number of points contained in the histogram
    """
    result = []
    for Sim in Sims:
        histkwargs = Sim.histkwargs
        tau_dict = Sim.tau_dict
        Nobjects = np.where(tau_dict[tau0_key] != -1)[0].size
        label = Sim.gasz0_label + ' (%d)'%Nobjects
        result.append(Patch(facecolor='None', edgecolor=histkwargs['color'],
                            lw=histkwargs['lw'], label=label))
    return result


In [ ]:
def plot_muz0_infalltimes_contours(Sims, savefig=False):
    """
    Plot muz0 vs infall time [tau_0] as contours for 
    each Sim in Sims. Sims should be a list of Sim classes.
    Optionally save the figure.
    Returns fig, ax
    """
    x_key = infall_key
    x_label = infall_label
    y_key = muz0_key
    y_label = 'Satellte to Host Mass Ratio \n' + r'[$\log_{10} M_{\star}^{\rm sat} / M_{\rm 200c}^{\rm host}$]'
    x_binwidth = 0.5
    y_binwidth = 0.1

    #y_key = Mstarz0_key
    #y_label = 'Satellite Stellar Mass \n' + r'[$M_{\star}^{\rm sat}(z=0) / {\rm M_\odot}$]'

    fig, ax = plt.subplots(figsize=(figsizewidth_column, figsizewidth_column / 1.1))
    ax.set_ylabel(y_label, **axislabel_kwargs)
    ax.set_xlabel(x_label, **axislabel_kwargs)
    ax = add_redshift_sincez5(ax, label=True)
    
    percentiles = [68, 84, 95, 97.5, 100]
    
    patches = []
    colors = ['tab:red', 'tab:blue']
    cmaps = ['Reds', 'Blues']
        
    for i, Sim in enumerate(Sims):
        color = colors[i]
        cmap = cmaps[i]

        tau_dict = Sim.tau_dict

        x = tau_dict[x_key]
        y = np.log10(tau_dict[y_key])
        
        mask = x > 0
        y = y[mask]
        x = x[mask]
        
        x_bins, x_bincents = ru.returnbins(x, x_binwidth)
        y_bins, y_bincents = ru.returnbins(y, y_binwidth)
        c = np.ones(x.size)
        bins = [x_bins, y_bins]
    
        statistic = scipy.stats.binned_statistic_2d(y, x, c, statistic='count', bins=[bins[1], bins[0]])
        extent = [bins[0][0], bins[0][-1], 
                  bins[1][0], bins[1][-1]]        
        
        vals = statistic[0]

        # plot contours based on percentiles
        levels = np.zeros(len(percentiles))
        for i, p in enumerate(percentiles):
            levels[i] = np.percentile(vals[vals > 0], p)
        
        patches.append(Patch(color=color, label=Sim.gasz0_label + '(%d)'%(x.size)))
        ax.contour(x_bincents, y_bincents, vals, levels=levels, cmap=cmap, extend=extent, alpha=1.0)
        ax.contourf(x_bincents, y_bincents, vals, levels=levels, cmap=cmap, extend=extent, alpha=0.5)

    ax.legend(handles=patches, title='%s %s'%(Sim.sim_label, Sim.taudict_label),
              title_fontsize='small', fontsize='x-small', loc='upper center')

    
    if (savefig):
        save_label = ''
        for Sim in Sims:
            save_label += Sim.save_label
            if Sim != Sims[-1]:
                save_label += '_'
        fname = '%s_muz0_infalltime_contours.pdf'%save_label
        for direc in direc_list:
            fig.savefig(direc + fname, bbox_inches='tight')

    return fig, ax

def plot_muz0_infalltimes(Sims, savefig=False):
    """
    Plots muz0 vs infall time [tau_0] for each Sim in Sims
    Sims should be a list of Sim classes. 
    Optionally saves the figure.
    Returns fig, ax
    """

    x_key = infall_key
    x_label = infall_label
    y_key = muz0_key
    y_label = muz0_label_words

    fig, ax = plt.subplots(figsize=(figsizewidth_column, figsizewidth_column / 1.1))
    ax.set_yscale('log')
    ax.set_ylabel(y_label, **axislabel_kwargs)
    ax.set_xlabel(x_label, **axislabel_kwargs)
    ax = add_redshift_sincez2(ax, label=True)
    
    percentiles = [16, 50, 84]

    for Sim in Sims:
        tau_dict = Sim.tau_dict

        x = tau_dict[x_key]
        y = tau_dict[y_key]

        mask = (x > 0) & (y > 0)

        x_plot = x[mask]
        y_plot = y[mask]

        ax.plot(x_plot, y_plot, label=(Sim.gasz0_label + ' (%d)'%x_plot.size), **Sim.plotkwargs)
        x_16 = np.percentile(x_plot, 16)
        x_84 = np.percentile(x_plot, 84)
        y_16 = np.percentile(y_plot, 16)
        y_84 = np.percentile(y_plot, 84)
        ax.fill_between([x_16, x_84], y_16, y_84, color=Sim.plotkwargs['color'], alpha=0.3)
        x_50 = np.median(x_plot)
        y_50 = np.median(y_plot)
        ax.axvline(x_50, color=Sim.plotkwargs['color'], **median_kwargs)
        print('x16, 50, 84: %.2g %.2g %.2g'%(x_16, x_50, x_84))
        print('y16, 50, 84: %.2e %.2e %.2e'%(y_16, y_50, y_84))
        """
        for percentile in percentiles:
            val = np.percentile(x_plot, percentile)
            if percentile == 50:
                kwargs = median_kwargs
            else:
                kwargs = vline_kwargs
            ax.axvline(val, color=Sim.plotkwargs['color'], **kwargs)
        """

    leg_kwargs = dict(ncol=1, bbox_to_anchor=(0.5, 1.25), loc='lower center', bbox_transform=ax.transAxes,
                      title='%s %s'%(Sim.sim_label, Sim.taudict_label), title_fontsize='medium', fontsize='small')

    ax.legend(**leg_kwargs)   
    
    # axs[1]: state the Mstar and M200c ranges for each sample
    textkwargs = dict(ha='left', va='top', ma='left', transform=ax.transAxes, fontsize='xx-small', **text_kwargs)
    xval = -0.2
    yval = 1.65
    ystep = 0.09
    ymstar = yval
    ym200c = ymstar - ystep * 3.1
    for i, Sim in enumerate(Sims):
        if i == 0:
            #ax.text(xval, yval, r'at $z=0:$', **textkwargs)
            ax.text(xval, ymstar, r'$\lg[M_{\star}^{\rm sat}] \in$', **textkwargs)
            ax.text(xval, ym200c, r'$\lg[M_{\rm 200c}^{\rm host}] \in$', **textkwargs)
        c = Sim.plotkwargs['color']
        mstarlolim = ru.floor_to_value(np.log10(Sim.tau_dict[Mstarz0_key].min()), 0.1)
        mstarhilim = ru.ceil_to_value(np.log10(Sim.tau_dict[Mstarz0_key].max()), 0.1)
        m200clolim = ru.floor_to_value(np.log10(Sim.tau_dict[M200cz0_key].min()), 0.1)
        m200chilim = ru.ceil_to_value(np.log10(Sim.tau_dict[M200cz0_key].max()), 0.1)
        ax.text(xval, ymstar - ystep*(i + 1), r'$(%.1f, %.1f)$'%(mstarlolim, mstarhilim), c=c, **textkwargs)
        ax.text(xval, ym200c - ystep*(i + 1), r'$(%.1f, %.1f)$'%(m200clolim, m200chilim), c=c, **textkwargs)
        
    if (savefig):
        save_label = ''
        for Sim in Sims:
            save_label += Sim.save_label
            if Sim != Sims[-1]:
                save_label += '_'
        fname = '%s_muz0_infalltime.pdf'%save_label
        for direc in direc_list:
            fig.savefig(direc + fname, bbox_inches='tight')

    return fig, ax

Sims = [TNGCluster_all_nogasz0,
        TNGCluster_all_gasz0]

#fig, ax = plot_muz0_infalltimes(Sims, savefig=savefig)
fig, ax = plot_muz0_infalltimes_contours(Sims, savefig=savefig)


"""
Sims = [TNGCluster_clean_nogasz0,
        TNGCluster_clean_gasz0]

#fig, ax = plot_muz0_infalltimes(Sims, savefig=savefig)
fig, ax = plot_muz0_infalltimes_contours(Sims, savefig=savefig)

Sims = [TNGCluster_PP_nogasz0,
        TNGCluster_PP_gasz0]

#fig, ax = plot_muz0_infalltimes(Sims, savefig=savefig)
fig, ax = plot_muz0_infalltimes_contours(Sims, savefig=savefig)
"""

In [ ]:


# code for adding median + error tick marks
def add_med_ticks(ax, dset, color='k', yval=0.90, errs=False, spread=True):
    """
    Add ticks for the median (and errors) plus 16/84th percentiles
    """
    
    med = np.median(dset)
    err = 1.25 * np.std(dset) / np.sqrt(dset.size)
    xlo = med - err
    xhi = med + err
    if xlo <= np.min(dset):
        xlo = np.min(dset)
    if xhi >= np.max(dset):
        xhi = np.max(dset)
    x16 = np.percentile(dset, 16)
    x84 = np.percentile(dset, 84)
    
    # median
    ax.plot([med, med], [yval, 1.0], lw=3, ls='-', color=color, marker='None', alpha=0.75,
            transform=transforms.blended_transform_factory(ax.transData, ax.transAxes))
    
    # error
    if errs:
        ax.fill_between([xlo, xhi], yval, 1.0, color=color, alpha=0.3,
                        transform=transforms.blended_transform_factory(ax.transData, ax.transAxes))
    
    # 16/84 percentiles
    if spread:
        ax.fill_between([x16, x84], yval, 1.0, color=color, alpha=0.3,
                        transform=transforms.blended_transform_factory(ax.transData, ax.transAxes))
                                    
    return ax


In [ ]:

m200c_lo = 14.8
m200c_hi = 15.2
mstar_lo = 10.9
mstar_hi = 11.1

def return_tau(Sim):
        tau_dict = Sim.tau_dict
        M200c = tau_dict[M200cz0_key]
        Mstar = tau_dict[Mstarz0_key]


        mask = ((M200c > 10.**(m200c_lo)) & (M200c < 10.**(m200c_hi)) &
                (Mstar > 10.**(mstar_lo)) & (Mstar < 10.**(mstar_hi)))
                
        tau = tau_dict[tau0_key][mask]
        return tau

tau0_nogas = return_tau(TNGCluster_all_nogasz0)
tau0_gas = return_tau(TNGCluster_all_gasz0)
tau0_tot = return_tau(TNGCluster_all_tot)

bins = ru.returnbins([0, 14], 1)[0]
hist_kwargs = dict(bins=bins, density=True, alpha=0.75)

fig, ax = plt.subplots(figsize=(figsizewidth_column, figsizeheight_column * 1.25))
ax.hist(tau0_tot, color='black', bins=bins, label=r'Total (%d)'%tau0_tot.size, histtype='step', density=True)
ax.hist(tau0_nogas, color='tab:red', **hist_kwargs, label=r'$M_{\rm gas}^{\rm sat} < 10^{9}\, {\rm M_\odot}$ (%d)'%tau0_nogas.size)
ax.hist(tau0_gas, color='tab:blue', **hist_kwargs, label=r'$M_{\rm gas}^{\rm sat} > 10^{9}\, {\rm M_\odot}$  (%d)'%tau0_gas.size)
ax.set_ylabel(r'PDF')
ax.set_xlabel(r'Cosmic Time at First Infall [Gyr]')
ax.legend(fontsize='small', loc='upper left')

title = (r'TNG-Cluster $z=0$ Satellites' + '\n' + 
         r'$M_{\rm 200c}^{\rm host}\sim10^{15}\, {\rm M_\odot},\ M_\star^{\rm sat}\sim 10^{11}\, {\rm M_\odot}$')
ax.set_title(title, fontsize='medium')
ax.set_ylim(0.0, 0.249)

dsets = [tau0_nogas, tau0_gas]
ks = ks_2samp(dsets[0], dsets[1]).pvalue
anderson = anderson_ksamp(dsets)
print(ks, anderson)

ax = add_med_ticks(ax, dsets[0], color='tab:red', yval=0.90, errs=True, spread=False)
ax = add_med_ticks(ax, dsets[1], color='tab:blue', yval=0.90, errs=True, spread=False)


if savefig:
    fname = 'L680n8192TNG_all_narrowbin_infalltimePDFs.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')


In [ ]:
m200c_lo = 14.8
m200c_hi = 15.2
mstar_lo = 10.0
mstar_hi = 12.5

def return_tau(Sim, tau0_key=tau0_key,
               m200c_lo=m200c_lo, m200c_hi=m200c_hi,
               mstar_lo=mstar_lo, mstar_hi=mstar_hi):
        
        tau_dict = Sim.tau_dict
        M200c = tau_dict[M200cz0_key]
        Mstar = tau_dict[Mstarz0_key]

        mask = ((M200c > 10.**(m200c_lo)) & (M200c < 10.**(m200c_hi)) &
                (Mstar > 10.**(mstar_lo)) & (Mstar < 10.**(mstar_hi)))
                
        tau = tau_dict[tau0_key][mask]
        return tau

tau0_m200clow = return_tau(TNGCluster_all_tot, m200c_lo=14.2, m200c_hi=14.7)
tau0_m200cmid = return_tau(TNGCluster_all_tot, m200c_lo=14.7, m200c_hi=15.0)
tau0_m200chigh = return_tau(TNGCluster_all_tot, m200c_lo=15.0, m200c_hi=15.4)
tau0_m200ctot = return_tau(TNGCluster_all_tot, m200c_lo=14.2, m200c_hi=15.4)

bins = ru.returnbins([0, 14], 1)[0]
hist_kwargs = dict(bins=bins, density=True, alpha=0.75, histtype='step')

colors_Mstar = ['bisque', 'orange', 'darkgoldenrod']

colors = ['purple', 'orange', 'yellow']

fig, ax = plt.subplots(figsize=(figsizewidth_column, figsizeheight_column * 1.25))
ax.hist(tau0_m200ctot, color='lightgray', bins=bins, label=r'Total (%d)'%tau0_m200ctot.size, density=True)
ax.hist(tau0_m200clow, color=colors[0], **hist_kwargs, label=r'$M_{\rm 200c}^{\rm host} \in 10^{14.3-14.7}\, {\rm M_\odot}$ (%d)'%tau0_m200clow.size)
ax.hist(tau0_m200cmid, color=colors[1], **hist_kwargs, label=r'$M_{\rm 200c}^{\rm host} \in 10^{14.7-15.0}\, {\rm M_\odot}$ (%d)'%tau0_m200cmid.size)
ax.hist(tau0_m200chigh, color=colors[2], **hist_kwargs, label=r'$M_{\rm 200c}^{\rm host} \in 10^{15.0-15.4}\, {\rm M_\odot}$ (%d)'%tau0_m200chigh.size)
ax.set_ylabel(r'PDF')
ax.set_xlabel(r'Cosmic Time at First Infall [Gyr]')
ax.legend(fontsize='small', loc='upper left')

title = (r'TNG-Cluster $z=0$ Satellites')
ax.set_title(title, fontsize='medium')
ax.set_ylim(0.0, 0.249)

dsets = [tau0_m200clow, tau0_m200cmid, tau0_m200chigh]
#ks = ks_2samp(dsets[0], dsets[1]).pvalue
#anderson = anderson_ksamp(dsets)
#print(ks, anderson)
colors = ['purple', 'orange', 'yellow']
ax = add_med_ticks(ax, dsets[0], color=colors[0], yval=0.90, errs=True, spread=False)
ax = add_med_ticks(ax, dsets[1], color=colors[1], yval=0.90, errs=True, spread=False)
ax = add_med_ticks(ax, dsets[2], color=colors[2], yval=0.90, errs=True, spread=False)

if savefig:
    fname = 'L680n8192TNG_all_infalltimePDFs_M200c.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')


In [ ]:

m200c_lo = 14.8
m200c_hi = 15.2
mstar_lo = 10.9
mstar_hi = 11.1

def return_tau(Sim, tau0_key=tau0_key,
               m200c_lo=m200c_lo, m200c_hi=m200c_hi,
               mstar_lo=mstar_lo, mstar_hi=mstar_hi):
        
        tau_dict = Sim.tau_dict
        M200c = tau_dict[M200cz0_key]
        Mstar = tau_dict[Mstarz0_key]

        mask = ((M200c > 10.**(m200c_lo)) & (M200c < 10.**(m200c_hi)) &
                (Mstar > 10.**(mstar_lo)) & (Mstar < 10.**(mstar_hi)))
                
        tau = tau_dict[tau0_key][mask]
        return tau

tau0_mstarlow = return_tau(TNGCluster_all_tot, mstar_lo=10.0, mstar_hi=10.5)
tau0_mstarmid = return_tau(TNGCluster_all_tot, mstar_lo=10.5, mstar_hi=11.5)
tau0_mstarhigh = return_tau(TNGCluster_all_tot, mstar_lo=11.5, mstar_hi=12.5)
tau0_mstartot = return_tau(TNGCluster_all_tot, mstar_lo=10.0, mstar_hi=12.5)

bins = ru.returnbins([0, 14], 1)[0]
hist_kwargs = dict(bins=bins, density=True, alpha=0.75, histtype='step')

colors_Mstar = ['bisque', 'orange', 'darkgoldenrod']


fig, ax = plt.subplots(figsize=(figsizewidth_column, figsizeheight_column * 1.25))
ax.hist(tau0_mstartot, color='lightgray', bins=bins, label=r'Total (%d)'%tau0_mstartot.size, density=True)
ax.hist(tau0_mstarlow, color='bisque', **hist_kwargs, label=r'$M_{\star}^{\rm sat} \in 10^{10.0-10.5}\, {\rm M_\odot}$ (%d)'%tau0_mstarlow.size)
ax.hist(tau0_mstarmid, color='orange', **hist_kwargs, label=r'$M_{\star}^{\rm sat} \in 10^{10.5-11.5}\, {\rm M_\odot}$ (%d)'%tau0_mstarmid.size)
ax.hist(tau0_mstarhigh, color='darkgoldenrod', **hist_kwargs, label=r'$M_{\star}^{\rm sat} \in 10^{11.5-12.5}\, {\rm M_\odot}$ (%d)'%tau0_mstarhigh.size)
ax.set_ylabel(r'PDF')
ax.set_xlabel(r'Cosmic Time at First Infall [Gyr]')
ax.legend(fontsize='small', loc='upper left')

title = (r'TNG-Cluster $z=0$ Satellites' + '\n' + 
         r'$M_{\rm 200c}^{\rm host}\sim10^{15}$')
ax.set_title(title, fontsize='medium')
ax.set_ylim(0.0, 0.249)

dsets = [tau0_mstarlow, tau0_mstarmid, tau0_mstarhigh]
#ks = ks_2samp(dsets[0], dsets[1]).pvalue
#anderson = anderson_ksamp(dsets)
#print(ks, anderson)

ax = add_med_ticks(ax, dsets[0], color='bisque', yval=0.90, errs=True, spread=False)
ax = add_med_ticks(ax, dsets[1], color='orange', yval=0.90, errs=True, spread=False)
ax = add_med_ticks(ax, dsets[2], color='darkgoldenrod', yval=0.90, errs=True, spread=False)

if savefig:
    fname = 'L680n8192TNG_all_infalltimePDFs_Mstar.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')


In [ ]:
def plot_colored2dhist(Sim, x_key=Mstarz0_key, y_key=MHGasz0_key,
                       x_label=r'$\log_{10}\left[M_{\star}^{\rm sat}(z=0) / {\rm M_\odot}\right]$',
                       y_label=r'$\log_{10}\left[M_{\rm HotGas}^{\rm sat}(z=0) / {\rm M_\odot}\right]$',
                       x_binwidth=0.25, y_binwidth=None,
                       c_key=M200cz0_key, c_log=True,
                       cmap='viridis', c_label=r'Median Host Mass [$M_{\rm 200c}^{\rm host} / {\rm M_\odot}$]',
                       title=True,
                       fig=None, ax=None,
                       add_contours=True, savefig=False):
    """
    For the given Sim, plot the 2D log-log x-y histogram colorcoded by the median c_key. 
    Optionally can overplot the contours denoting the population.
    """
    if c_log: 
        norm = mpl.colors.LogNorm()
    else:
        norm = None

    if y_binwidth is None:
        y_binwidth = x_binwidth
        
    tau_dict = Sim.tau_dict
    indices = (tau_dict[x_key] > 0) & (tau_dict[y_key] > 0)
    x = np.log10(tau_dict[x_key][indices])
    y = np.log10(tau_dict[y_key][indices])
            
    x_bins, x_bincents = ru.returnbins(x, x_binwidth)
    y_bins, y_bincents = ru.returnbins(y, y_binwidth)
    c = tau_dict[c_key][indices]
    bins = [x_bins, y_bins]
    
    statistic = scipy.stats.binned_statistic_2d(y, x, c, statistic='median', bins=[bins[1], bins[0]])
    
    extent = [bins[0][0], bins[0][-1], 
              bins[1][0], bins[1][-1]]
    
    if (not fig) and (not ax):
        fig, ax = plt.subplots()

    h = ax.imshow(statistic[0], norm=norm, origin='lower', aspect='auto', cmap=cmap,
                  extent=extent)
        
    if c_label:
        cbar = fig.colorbar(h, ax=ax, label=c_label)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    if title:
        ax.set_title(Sim.plot_label, fontsize='medium')
    
    if add_contours:
        statistic = scipy.stats.binned_statistic_2d(y, x, np.ones(x.size), statistic='count',
                                                    bins=[bins[1], bins[0]])

        vals = statistic[0]

        # plot contours based on percentiles
        percentiles = [16, 50, 84, 95]
        levels = np.zeros(len(percentiles))
        for i, p in enumerate(percentiles):
            levels[i] = np.percentile(vals[vals > 0], p)
        
        colors = ['lightgray', 'darkgray', 'gray', 'black']
        ax.contour(x_bincents, y_bincents, vals, levels=levels, colors=colors, extend=extent)

    if (savefig):
        fname = 'L680n8192TNG_all_gasz0_%s-%s-%s.pdf'%(x_key, y_key, c_key)
        for direc in direc_list:
            fig.savefig(direc + fname, bbox_inches='tight')
            
    if not c_label:
        return fig, ax, h
            
    return fig, ax

In [ ]:
Sim = TNGCluster_all_z0_gasz0
fig, ax =  plot_colored2dhist(Sim, cmap='plasma', savefig=savefig)

In [ ]:
"""fig, ax = plot_colored2dhist(Sim, c_key=time_since_infall_key, c_log=False,
                             cmap='Oranges_r', c_label=r'Median Time Since Infall [Gyr]',
                             savefig=savefig)"""

In [ ]:
fig, ax = plot_colored2dhist(Sim, c_key=HCDnz0_key,
                             cmap='Purples', c_label=r'Median Host Centric Distance [$d_{\rm sat}^{\rm host}/ R_{\rm 200c}$]',
                             savefig=savefig)

In [ ]:
Sim = TNGCluster_all_z0_gasz0
x_key = M200cz0_key
x_label = r'$\log_{10}\left[M_{\rm 200c}^{\rm host} / {\rm M_\odot}\right]$'
y_key = Mstarz0_key
y_label = r'$\log_{10}\left[M_{\star}^{\rm sat} / {\rm M_\odot}\right]$'
x_binwidth = 0.1
y_binwidth = 0.25
c_key = MGasz0_key
c_log = True
cmap = 'Purples'
c_label = r'Sat. Gas Mass [${\rm M_\odot}$]'
add_contours = False
savefig = False
fig, ax = plt.subplots()
fig, ax = plot_colored2dhist(Sim, x_key, y_key, x_label, y_label,
                             x_binwidth, y_binwidth, c_key, c_log,
                             cmap, c_label, add_contours=add_contours,
                             fig=fig, ax=ax,
                             savefig=savefig)



        

In [ ]:
# make a success rate plot, where the color shows the faction of gaseous satellites 
Sim = TNGCluster_all_z0
tau_dict = Sim.tau_dict
c_key = MGasz0_key
weights = np.zeros(tau_dict[x_key].size, dtype=int)
weights[tau_dict[c_key] > 0] = 1

w = weights

x = np.log10(tau_dict[x_key])
y = np.log10(tau_dict[y_key])

x_binwidth = 0.1
y_binwidth = 0.2

x_bins, x_bincents = ru.returnbins(x, x_binwidth)
y_bins, y_bincents = ru.returnbins(y, y_binwidth)

bins = [y_bins, x_bins]
H, xedges, yedges = np.histogram2d(y, x, bins=bins, weights=w)
H2, _, _ = np.histogram2d(y, x, bins=bins)
extent = [yedges[0], yedges[-1], xedges[0], xedges[-1]]

vals = H / H2

fig, ax = plt.subplots(figsize=(figsizewidth_column, figsizeheight_column))

h = ax.imshow(vals, extent=extent, norm=mpl.colors.LogNorm(), origin='lower', aspect='auto')

cbar= fig.colorbar(h, ax=ax, label=r'Frac. of Satellites w/ Gas')

ax.set_ylabel(Mstarz0_label)
ax.set_xlabel(M200cz0_label)

ax.set_title(r'TNG-Cluster All (%d)'%(x.size))

if savefig:
    fname = 'L680n8192TNG_all_tot_Mstarz0_M200cz0_gasretentionrate.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')

        

In [ ]:
# make a success rate plot, where the color shows the faction of gas-rich satellites 
# this time require Mgas > 10e9
Sim = TNGCluster_all_z0
add_fgas_dset(Sim)
tau_dict = Sim.tau_dict
c_key = MGasz0_key
weights = np.zeros(tau_dict[x_key].size, dtype=int)
mask = (tau_dict[c_key] > 0) & (tau_dict[fgas_z0_key] > 1.0e-1)
weights[mask] = 1

w = weights

x = np.log10(tau_dict[x_key])
y = np.log10(tau_dict[y_key])

x_binwidth = 0.1
y_binwidth = 0.2

x_bins, x_bincents = ru.returnbins(x, x_binwidth)
y_bins, y_bincents = ru.returnbins(y, y_binwidth)

bins = [y_bins, x_bins]
H, xedges, yedges = np.histogram2d(y, x, bins=bins, weights=w)
H2, _, _ = np.histogram2d(y, x, bins=bins)
extent = [yedges[0], yedges[-1], xedges[0], xedges[-1]]

vals = H / H2

fig, ax = plt.subplots(figsize=(figsizewidth_column, figsizeheight_column))

h = ax.imshow(vals, extent=extent, norm=mpl.colors.LogNorm(), origin='lower', aspect='auto')

cbar= fig.colorbar(h, ax=ax, label=r'Frac. of Satellites w/ Gas')

ax.set_ylabel(Mstarz0_label)
ax.set_xlabel(M200cz0_label)

ax.set_title(r'TNG-Cluster All (%d)'%(x.size))

if savefig:
    fname = 'L680n8192TNG_all_tot_Mstarz0_M200cz0_gasretentionrate.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')



In [ ]:
add_fgas_dset(Sim)
Sim.tau_dict.keys()

In [ ]:
Sim = TNGCluster_all_z0
mass_binwidth = 0.2

tau_dict = Sim.tau_dict
Mstar = tau_dict[Mstarz0_key].copy()
MHGas = tau_dict[MHGasz0_key].copy()
MGas = tau_dict[MGasz0_key].copy()
MCGas = tau_dict[MCGasz0_key].copy()

Mstar_bins, Mstar_bincents = ru.returnlogbins(Mstar, mass_binwidth)

sort_indices = np.digitize(Mstar, Mstar_bins)
gaslessMgas_frac = np.zeros(Mstar_bincents.size, Mstar_bincents.dtype) - 1
NgaslessMgas = gaslessMgas_frac.copy()
gaslessMHGas_frac = gaslessMgas_frac.copy()
NgaslessMHgas = gaslessMgas_frac.copy()
gaslessMCGas_frac = gaslessMgas_frac.copy()
NgaslessMCgas = gaslessMgas_frac.copy()

for i, bincent in enumerate(Mstar_bincents):
    subset_indices = sort_indices == (i+1)
    Mgas_subset = MGas[subset_indices]
    if Mgas_subset.size == 0:
        continue
    gaslessMgas_frac[i] = Mgas_subset[Mgas_subset > 1.0e9].size / Mgas_subset.size 
    NgaslessMgas[i] = Mgas_subset[Mgas_subset > 1.0e9].size 
    Mhgas_susbset = MHGas[subset_indices]
    gaslessMHGas_frac[i] = Mhgas_susbset[Mhgas_susbset > 1.0e9].size / Mhgas_susbset.size
    NgaslessMHgas[i] = Mhgas_susbset[Mhgas_susbset > 1.0e9].size 
    Mcgas_subset = MCGas[subset_indices]
    gaslessMCGas_frac[i] = Mcgas_subset[Mcgas_subset > 1.0e9].size / Mcgas_subset.size
    NgaslessMCgas[i] = Mcgas_subset[Mcgas_subset > 1.0e9].size 

indices = gaslessMgas_frac != -1
fig, axs = plt.subplots(2, 1, figsize=(figsizewidth_column, figsizeheight_column * 2))

# top axis: fraction of satellites
ax = axs[0]
ax.plot(Mstar_bincents[indices], gaslessMgas_frac[indices], color='tab:purple', marker='None', ls='-', lw=4, label=r'$M_{\rm AllGas}^{\rm sat} > 10^{9}\, {\rm M_\odot}$')
ax.plot(Mstar_bincents[indices], gaslessMHGas_frac[indices], color='tab:red', marker='None', ls='--', label=r'$M_{\rm HotGas}^{\rm sat} > 10^{9}\, {\rm M_\odot}$')
ax.plot(Mstar_bincents[indices], gaslessMCGas_frac[indices], color='tab:blue', marker='None', ls='-.', label=r'$M_{\rm ColdGas}^{\rm sat} > 10^{9}\, {\rm M_\odot}$')
ax.set_xscale('log')
ax.set_title('TNG-Cluster All (%d)'%Mstar.size)
ax.set_xlabel(Mstarz0_label)
ax.set_ylabel('Fraction of Satellites')
ax.legend(loc='upper left', title='Satellites with')

# bottom axis: number of satellites
ax = axs[1]
ax.plot(Mstar_bincents[indices], NgaslessMgas[indices], color='tab:purple', marker='None', ls='-', ds='steps-mid', lw=3)
ax.fill_between(Mstar_bincents[indices], NgaslessMgas[indices], color='tab:purple',step='mid', alpha=0.7)
ax.plot(Mstar_bincents[indices], NgaslessMHgas[indices], color='tab:red', marker='None', ls='--', ds='steps-mid', lw=2)
ax.fill_between(Mstar_bincents[indices], NgaslessMHgas[indices], color='tab:red',step='mid', hatch='//', facecolor='none')
ax.plot(Mstar_bincents[indices], NgaslessMCgas[indices], color='tab:blue', marker='None', ls='--', ds='steps-mid', lw=2)
ax.fill_between(Mstar_bincents[indices], NgaslessMCgas[indices], color='tab:blue',step='mid', hatch='\\\\', facecolor='none')
ax.set_yscale('log')
ax.set_ylabel(r'Number of Satellites')
ax.set_xlabel(Mstarz0_label)
ax.set_xscale('log')
#ax.legend(loc='lower left', title='Satellites with')
savefig=False
if savefig:
    fname = '%s_fgassat-mstarz0.pdf'%(Sim.save_label)
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')
        

In [ ]:
fig = plt.figure(figsize=(figsizewidth, figsizeheight*1.2))

gs = gridspec.GridSpec(2, 2, figure=fig, height_ratios=(1,1))

ax = fig.add_subplot(gs[0,0])
# make a success rate plot, where the color shows the faction of gaseous satellites 
Sim = TNGCluster_all_z0
tau_dict = Sim.tau_dict
weights = np.zeros(tau_dict[x_key].size, dtype=int)
weights[tau_dict[c_key] > 0] = 1

w = weights

x = np.log10(tau_dict[x_key])
y = np.log10(tau_dict[y_key])

x_binwidth = 0.1
y_binwidth = 0.2

x_bins, x_bincents = ru.returnbins(x, x_binwidth)
y_bins, y_bincents = ru.returnbins(y, y_binwidth)

bins = [y_bins, x_bins]
H, xedges, yedges = np.histogram2d(y, x, bins=bins, weights=w)
H2, _, _ = np.histogram2d(y, x, bins=bins)
extent = [yedges[0], yedges[-1], xedges[0], xedges[-1]]

vals = H / H2

h = ax.imshow(vals, extent=extent, norm=mpl.colors.LogNorm(), origin='lower', aspect='auto')

cbar = fig.colorbar(h, ax=ax, label=r'Fraction of Satellites with $M_{\rm gas}^{\rm sat} > 10^9\, {\rm M_\odot}$', location='top', orientation='horizontal', pad=0.05)

ax.set_ylabel(r'Satellite Mass [$\log_{10} M_{\star}^{\rm sat} / {\rm M_\odot}$]')
ax.set_xlabel(r'Cluster Mass [$\log_{10} M_{\rm 200c}^{\rm host} / {\rm M_\odot}$]')

#ax.set_title(r'TNG-Cluster All (%d)'%(x.size))


ax = fig.add_subplot(gs[0,1])

Sim = TNGCluster_all_z0_gasz0
x_key = M200cz0_key
x_label = r'Cluster Mass [$\log_{10} M_{\rm 200c}^{\rm host} / {\rm M_\odot}$]'
y_key = Mstarz0_key
y_label = r'Satellite Mass [$\log_{10} M_{\star}^{\rm sat} / {\rm M_\odot}$]'
x_binwidth = 0.1
y_binwidth = 0.25
c_key = MGasz0_key
c_log = True
cmap = 'Purples'
c_label = False
add_contours = False
savefig = False

fig, ax, h = plot_colored2dhist(Sim, x_key, y_key, x_label, y_label,
                             x_binwidth, y_binwidth, c_key, c_log,
                             cmap, c_label, add_contours=add_contours,
                             fig=fig, ax=ax, title=False,
                             savefig=savefig)

cbar = fig.colorbar(h, ax=ax, label=r'Median Satellite Gas Mass [$M_{\rm gas}^{\rm sat} / {\rm M_\odot}$]', location='top', orientation='horizontal', pad=0.05)
ax.text(0.1, 0.05, r'only $M_{\rm gas}^{\rm sat} > 10^9\, {\rm M_\odot}$ (%d)'%(Sim.tau_dict[x_key].size),
        ha='left', va='bottom', transform=ax.transAxes)


ax = fig.add_subplot(gs[1,0])


# left axis: fraction of satellites
ax.plot(Mstar_bincents[indices], gaslessMgas_frac[indices], color='tab:purple', marker='None', ls='-', lw=4, label=r'$M_{\rm AllGas}^{\rm sat} > 10^9\, {\rm M_\odot}$ (%d)'%(NgaslessMgas[indices].sum()))
ax.plot(Mstar_bincents[indices], gaslessMHGas_frac[indices], color='tab:red', marker='None', ls='--', label=r'$M_{\rm HotGas}^{\rm sat} > 10^9\, {\rm M_\odot}$ (%d)'%(NgaslessMHgas[indices].sum()))
ax.plot(Mstar_bincents[indices], gaslessMCGas_frac[indices], color='tab:blue', marker='None', ls='-.', label=r'$M_{\rm ColdGas}^{\rm sat} > 10^9\, {\rm M_\odot}$ (%d)'%(NgaslessMCgas[indices].sum()))
ax.set_xscale('log')
ax.set_xlabel(r'Satellite Stellar Mass [$M_{\star}^{\rm sat} / {\rm M_\odot}$]')
ax.set_ylabel('Fraction of Satellites')
ax.set_yscale('log')
ax.set_ylim(1.0e-2, 7)
ax.axhline(1, marker='None', ls='-', lw=1, alpha=0.9, zorder=1)
ax.axhline(0.1, marker='None', ls='-', lw=0.75, alpha=0.6, zorder=1)
ax.legend(loc='upper left', title=r'Satellites with', fontsize='small', frameon=True, framealpha=1.0)

# right axis: number of satellites
ax = fig.add_subplot(gs[1,1])
ax.plot(Mstar_bincents[indices], NgaslessMgas[indices], color='tab:purple', marker='None', ls='-', ds='steps-mid', lw=3)
ax.fill_between(Mstar_bincents[indices], NgaslessMgas[indices], color='tab:purple',step='mid', alpha=0.7)
ax.plot(Mstar_bincents[indices], NgaslessMHgas[indices], color='tab:red', marker='None', ls='--', ds='steps-mid', lw=2)
ax.fill_between(Mstar_bincents[indices], NgaslessMHgas[indices], color='tab:red',step='mid', hatch='//', facecolor='none')
ax.plot(Mstar_bincents[indices], NgaslessMCgas[indices], color='tab:blue', marker='None', ls='--', ds='steps-mid', lw=2)
ax.fill_between(Mstar_bincents[indices], NgaslessMCgas[indices], color='tab:blue',step='mid', hatch='\\\\', facecolor='none')
ax.set_yscale('log')
ax.set_ylabel(r'Number of Satellites')
ax.set_xlabel(r'Satellite Stellar Mass [$M_{\star}^{\rm sat} / {\rm M_\odot}$]')
ax.set_xscale('log')

fig.suptitle(r'TNG-Cluster: %d halos, %d FoF satellites with $M_{\star}^{\rm sat} > 10^9\, {\rm M_\odot}$ at $z=0$'%(352, x.size))

if savefig:
    fname = 'L680n8192TNG_GaseousSatellitesFraction_multipanel.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')

In [ ]:
### investigate the evolution of the gas density radial profiles
# start by loading the GRP dictionary for a few z=0 gasless objects 
# start with low mass first-infalling objects
TNGCluster_all_new = TNGSim(sim, all_key, tot_key)
TNGCluster_all_new.tau_dict = load_taudict_new(TNGCluster_all_new, return_taudict_fname(TNGCluster_all_new))

Sim = TNGCluster_all
tau_dict = Sim.tau_dict

subfindIDs = tau_dict['SubfindID']

missing_groups = []

# load input file into a dictionary
def load_grpdict():
    result = {}
    infname = 'subfind_L680n8192TNG_branches_all.hdf5'
    with h5py.File('../Output/L680n8192TNG_subfindGRP_20230916/' + infname, 'r') as f:
        for group_key in f.keys():
            result[group_key] = {}
            for dset_key in f[group_key].keys():
                result[group_key][dset_key] = f[group_key][dset_key][:]
        f.close()
    
    return result

grp_dict = load_grpdict()
grp_dict_keys = np.array(list(grp_dict.keys()))



In [ ]:
### investigate the evolution of the gas density radial profiles
# start by loading the GRP dictionary for a few z=0 gasless objects 
# start with low mass first-infalling objects

# load input file into a dictionary
def load_grpdict_new():
    result = {}
    infname = 'subfind_L680n8192TNG_branches_all.hdf5'
    with h5py.File('../Output/L680n8192TNG_subfindGRP/' + infname, 'r') as f:
        for group_key in f.keys():
            result[group_key] = {}
            for dset_key in f[group_key].keys():
                result[group_key][dset_key] = f[group_key][dset_key][:]
        f.close()
    
    return result

grp_dict_new = load_grpdict()
grp_dict_new_keys = np.array(list(grp_dict_new.keys()))



In [ ]:
print(grp_dict[grp_dict_keys[0]].keys())

In [ ]:
# find the grp_dict_keys where tau_100^AllGas is reached
Sim = TNGCluster_all_new
tau_dict = Sim.tau_dict

tau100 = tau_dict[tau100_key]

mask = tau100 > 0

subfindIDs_tau100 = tau_dict['SubfindID'][mask]
grp_dict_keys_tau100 = []
for subfindID in subfindIDs_tau100:
    grp_dict_keys_tau100.append('%08d'%subfindID)

TNGCluster_all_gasz0_new = TNGSim(sim, all_key, tot_key)
TNGCluster_all_gasz0_new.tau_dict = load_taudict_new(TNGCluster_all_gasz0_new, return_taudict_fname(TNGCluster_all_gasz0_new))

subfindIDs_gasz0 = TNGCluster_all_gasz0_new.tau_dict['SubfindID'].copy()
grp_dict_keys_gasz0 = []
for subfindID in subfindIDs_gasz0:
    grp_dict_keys_gasz0.append('%08d'%subfindID)


In [ ]:
def plot_SGRPevolution(grp_dict, grp_dict_key, savefig=False, clf=False):
    """
    Plot in two panels the evolution of hot and cold gas radial profiles.
    Start at the time of peak hot and cold gas mass and plot until there 
    is no more gas. Include some extra info in each panel about the 
    given subhalo.
    """
    
    group = grp_dict[grp_dict_key]
    
    fig, axs = plt.subplots(2, 1, figsize=(figsizewidth, figsizeheight*1.5))
    
    gastypes = ['HotGas', 'ColdGas']
    cmaps = ['Reds', 'Blues']
    for i, gastype in enumerate(gastypes):
        ax = axs[i]
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlim(3.0e-1, 1.0e3)
        ax.set_ylim(1.0e-2, 1.0e8)
        ax.set_xlabel(r'Galacto-centric distance $[r / 2R_{\star}]$')
        ax.set_ylabel(r'$\rho_{\rm s1}^{\rm sub}(r + {\rm d}r)\, [{\rm M_\odot\, kpc^{-3}}]$'.replace('s1', gastype))
        
        start = np.where(group['tau_medpeak_%s'%gastype] == 0)[0][0]
        end = np.where(group['tau_medpeak_%s'%gastype] == 100)[0]
        if end.size == 0:
            return
        end = end.max()
        time_at_peak = group['CosmicTime'][start]
        
        xs = []
        ys = []
        cs = []
        
        # loop over the snapshots
        for index in range(start, end, -1):
            x = group['radii'][index]
            y = group['Subhalo%sDensityShells'%gastype][index]
            
            plot_indices = y > 0

            xs.append(x[plot_indices])
            ys.append(y[plot_indices])

            cs.append(group['CosmicTime'][index] - time_at_peak)  
        # end loop over snapshots
        kwargs = dict(path_effects=[pe.Stroke(linewidth=6, foreground='k'), pe.Normal()],
                      cmap=cmaps[i])
    
        lc = ru.multiline(xs, ys, cs, ax=ax, **kwargs)
        
        # add the colorbar 
        loc = 'lower left'
        cax = inset_axes(ax, width='50%', height='5%', loc=loc) 
        cbar = plt.colorbar(lc, cax=cax, orientation='horizontal')
        cbar.set_label(r'Time since max $M_{\rm s1}^{\rm sat}$ [Gyr]'.replace('s1', gastype), labelpad=5, fontsize='small')
        cbar.ax.tick_params(labelsize='x-small')
        cax.minorticks_off()
        if 'lower' in loc:
            cax.xaxis.set_label_position('top')
            cax.xaxis.set_ticks_position('top')
            
        # add some extra info about the galaxy
        ax.text(3.5e-1, 1.0e1, (r'Cosmic Time at max $M_{\rm s1}^{\rm sat}$ = %.1f [Gyr]'%time_at_peak).replace('s1', gastype),
                ha='left', va='bottom', fontsize='small')
        
        if i == 0:
            ax.set_title('TNG Cluster subfindID %s at snap 099'%grp_dict_key)
            ax.text(9.0e2, 3.0e7, r'$M_{\star}^{\rm sat}(z=0) = s1\, [{\rm M_\odot}]$'.replace('s1',ru.latex_float(group['Subhalo_Mstar_Rgal'][0], 1)),
                    va='top', ha='right', fontsize='small')
            ax.text(9.0e2, 3.0e6, r'$M_{\rm 200c}^{\rm host}(z=0) = s1\, [{\rm M_\odot}]$'.replace('s1',ru.latex_float(group['HostGroup_M_Crit200'][0], 1)),
                    va='top', ha='right', fontsize='small')            
        if i == 1:
            infall_index = np.where(group['tau_infall_%s'%gastype] == 0)[0][0]
            infall_time = group['CosmicTime'][infall_index]
            ax.text(9.0e2, 3.0e7, 'Infall Time: %.1f [Gyr]'%infall_time, va='top', ha='right', fontsize='small')
            peakmass_index = np.where(group['tau_medpeak_Mass'] == 0)[0][0]
            peakmass_time = group['CosmicTime'][peakmass_index]
            ax.text(9.0e2, 3.0e6, 'Time of max mass: %.1f [Gyr]'%peakmass_time, va='top', ha='right', fontsize='small')
            
            
    if savefig:
        fname = 'L680n8192TNG_snap099_subfindID%s_SGRPevolution.pdf'%grp_dict_key
        direc = '../Figures/TNGCluster/SGRPevolution/'
        fig.savefig(direc + fname, bbox_inches='tight')
        
    if clf:
        plt.close(fig)
        return
            
    # finish loop over gas types
    return fig, axs



In [ ]:
for grp_dict_key in ['%08d'%(59)]:
    plot_SGRPevolution(grp_dict, grp_dict_key, savefig=False, clf=False)

In [ ]:
rmin = 1
rmax = 1.0e3
binwidth_log = 0.1
_, radii_phys_bincents = ru.returnlogbins([rmin, rmax], binwidth_log)
radii_norm_bincents = grp_dict[grp_dict_keys[0]]['radii'][0,:] / grp_dict[grp_dict_keys[0]]['Subhalo_Rgal'][0]

bad_keys = []

def return_stacked_dict(grp_dict_keys, dset_key='SubhaloGasMassShells', time='z0',
                        return_all_profiles=False, return_color_dset=False):
    """
    Given the grp_dict_keys and the implicit grp_dict, stack the dset_key for all of the
    keys at the given time, which must match be either z0 or a tau definition, such as tau_infall_mass
    """
        
    result_dict = {}
 
    # initalize the outputs
    result_dict['50_phys'] = np.zeros((len(grp_dict_keys), len(radii_phys_bincents)), dtype=float) - 1.
    result_dict['16_phys'] = result_dict['50_phys'].copy()
    result_dict['84_phys'] = result_dict['50_phys'].copy()
    result_dict['50_norm'] = np.zeros((len(grp_dict_keys), len(radii_norm_bincents)), dtype=float) - 1.
    result_dict['16_norm'] = result_dict['50_norm'].copy()
    result_dict['84_norm'] = result_dict['50_norm'].copy()    
    phys = np.zeros((len(grp_dict_keys), len(radii_phys_bincents)), dtype=float) - 1.
    norm = np.zeros((len(grp_dict_keys), len(radii_norm_bincents)), dtype=float) - 1.    

    if return_color_dset:
        if return_color_dset in grp_dict[grp_dict_keys[0]].keys():
            color_dset = np.zeros(len(grp_dict_keys), dtype=grp_dict[grp_dict_keys[0]][return_color_dset].dtype) - 1
        else:
            raise KeyError('return_color_dset %d not recognized.'%return_color_dset)

    for index, grp_dict_key in enumerate(grp_dict_keys):
        
        group = grp_dict[grp_dict_key]

        if time == 'z0':
            time_index = 0
        else:
            tau = group[time]
            time_index = np.where(tau == 0)[0][0]

        masses = group[dset_key][time_index]
        radii = group['radii'][time_index]
        volumes = group['vol_shells'][time_index]
        rgal = group['Subhalo_Rgal'][time_index]

        mask = masses > 3.0e6

        if mask[mask].size < 2:
            continue 
        
        # interpolate the non0 masses, and write the values outside of the bounds as 0
        massfunc_phys = interp1d(radii[mask], masses[mask], bounds_error=False, fill_value=0)
        densities_phys = massfunc_phys(radii) / volumes
        densities_phys_model = interp1d(radii, densities_phys, bounds_error=False, fill_value=0)
        phys[index,:] = densities_phys_model(radii_phys_bincents) 

        radii_norm = radii / rgal
        massfunc_norm = interp1d(radii_norm[mask], masses[mask], bounds_error=False, fill_value=0)
        densities_norm = massfunc_norm(radii_norm) / volumes
        densities_norm_model = interp1d(radii_norm, densities_norm, bounds_error=False, fill_value=0)
        norm[index,:] = densities_norm_model(radii_norm_bincents)

        if return_color_dset:
            color_dset[index] = group[return_color_dset][time_index]

    # finish loop of indices, save final results
    norm = np.ma.masked_values(norm, -1)
    phys = np.ma.masked_values(phys, -1)
    
    result_dict['50_phys'] = np.median(phys, axis=0)
    result_dict['16_phys'] = np.percentile(phys, 16, axis=0)
    result_dict['84_phys'] = np.percentile(phys, 84, axis=0) 
    result_dict['50_norm'] = np.median(norm, axis=0)
    result_dict['16_norm'] = np.percentile(norm, 16, axis=0)
    result_dict['84_norm'] = np.percentile(norm, 84, axis=0) 
    result_dict['Ngal'] = len(norm)
    
    if return_all_profiles:
        if return_color_dset:
            return result_dict, norm, phys, color_dset
        else:
            return result_dict, norm, phys
    else:
        if return_color_dset:
            return result_dict, color_dset
        else:
            return result_dict

In [ ]:

Sim = TNGCluster_all_gasz0
tau_dict_gasz0 = Sim.tau_dict

def plot_SGRP_stack(fig, ax, tau_dict=tau_dict_gasz0, Mstar_cent=11.0, Mstar_binwidth=0.2, M200c_cent=15, M200c_binwidth=0.2):
    """
    calcuulate the stack SGRP and plot onto the ax for the given Mstar and M200c range
    Mstar_cent, Mstar_binwidth, M200c_cent, M200c_binwidth should all be log
    """

    Mstar_binedges = [10.**(Mstar_cent - Mstar_binwidth/2.), 10.**(Mstar_cent + Mstar_binwidth/2.)]
    M200c_binedges = [10.**(M200c_cent - M200c_binwidth/2.), 10.**(M200c_cent + M200c_binwidth/2.)]

    Mstarz0 = tau_dict[Mstarz0_key]
    M200cz0 = tau_dict[M200cz0_key]

    mask = ((Mstarz0 > Mstar_binedges[0]) & (Mstarz0 < Mstar_binedges[1]) & 
            (M200cz0 > M200c_binedges[0]) & (M200cz0 < M200c_binedges[1]))

    subfindIDs = tau_dict['SubfindID'][mask]
    grp_dict_keys_bin = []
    for subfindID in subfindIDs:
        grp_dict_keys_bin.append('%08d'%subfindID)

    stacked_dict_allgas = return_stacked_dict(grp_dict_keys_bin, dset_key='SubhaloGasMassShells', time='z0')
    stacked_dict_coldgas = return_stacked_dict(grp_dict_keys_bin, dset_key='SubhaloColdGasMassShells', time='z0')
    stacked_dict_hotgas = return_stacked_dict(grp_dict_keys_bin, dset_key='SubhaloHotGasMassShells', time='z0')
    stacked_dict_allgas_infall = return_stacked_dict(grp_dict_keys_bin, dset_key='SubhaloGasMassShells', time='tau_infall_Gas')

    radii = 'norm'
    x_label = r'Galacto-Centric Distance [$ r / R_{\rm half,\star}$]'
    radii_bincents = grp_dict[grp_dict_keys_bin[0]]['radii'][0,:] / grp_dict[grp_dict_keys_bin[0]]['Subhalo_Rgal'][0]

    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(x_label, fontsize='small')
    #ax.set_ylabel(r'Gas Densitity [${\rm M_\odot\, kpc^{-3}}$]', fontsize='small)

    stacked_dicts = [stacked_dict_allgas_infall,
                    stacked_dict_allgas,
                    stacked_dict_coldgas,
                    stacked_dict_hotgas]

    colors = ['black',
            'tab:purple',
            'tab:blue', 
            'tab:red']

    labels = [r'All Gas at infall as centrals',
            r'All Gas at $z=0$ as satellites',
            r'Cold Gas at $z=0$ as satellites',
            r'Hot Gas at $z=0$ as satellites']

    for i, stacked_dict in enumerate(stacked_dicts):

        vals_50s = stacked_dict['50_%s'%radii]

        mask = vals_50s > 0

        color = colors[i]
        label = labels[i]
        ax.plot(radii_bincents[mask], vals_50s[mask], '-', color=color, label=label)
        #if i != 0:
        #    ax.fill_between(radii_bincents[mask], vals_16s[mask], vals_84s[mask], color=color, alpha=0.2)

    # finish loop over dicts  
    text = r'$M_\star^{\rm sat} \sim 10^{%.1f}\, {\rm M_\odot}\ (%d)$'%(Mstar_cent, stacked_dicts[0]['Ngal'])
    if abs(Mstar_cent - 10.75) < 1.0e-3:
        text = r'$M_\star^{\rm sat} \sim 10^{%.2f}\, {\rm M_\odot}\ (%d)$'%(Mstar_cent, stacked_dicts[0]['Ngal'])
    ax.text(0.95, 0.95, text,
            ha='right', va='top', transform=ax.transAxes, fontsize='small')
    #ax.legend(title=r'$M_\star^{\rm sat} \sim 10^{%.1f}\, {\rm M_\odot}\ (%d)$'%(Mstar_cent, stacked_dicts[0]['Ngal']), ncol=1)
    ax.set_xlim(5.0e-1, 80)
    ax.set_ylim(1.0e0, 2e7)
    ax = add_ISM_CGM_line(ax)
    return fig, ax


def add_ISM_CGM_line(ax):
    """ add the demarcating line between the ISM and CGM to ax at xval """

    ax.axvline(2.0, c='black', ls='--', marker='None')

    trans = transforms.blended_transform_factory(ax.transData, ax.transAxes)
    
    annotate_kwargs = dict(xytext=((1.9),(0.05)), xycoords=trans,
                           arrowprops=dict(facecolor='tab:gray', shrink=0.05),
                           fontsize='medium', ha='right', va='center', c='black')
    ax.annotate('', (0.5, 0.05), **annotate_kwargs)
    ax.text(1.9, 0.075, r'ISM', transform=trans, ha='right', va='bottom', c='black')

    annotate_kwargs = dict(xytext=((2.1),(0.05)), xycoords=trans,
                           arrowprops=dict(facecolor='tab:pink', shrink=0.05),
                           fontsize='medium', ha='left', va='center', c='m')
    ax.annotate('', (10.0, 0.05), **annotate_kwargs)
    ax.text(2.25, 0.075, r'CGM', transform=trans, ha='left', va='bottom', c='m')

    return ax



In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(figsizewidth, figsizeheight/1.95), sharey=True)
Mstar_cents = [10.049, 10.75, 11.5]
Mstar_binwidths = [0.1, 0.1, 0.3]
for i, Mstar_cent in enumerate(Mstar_cents):
    Mstar_binwidth = Mstar_binwidths[i]
    fig, ax = plot_SGRP_stack(fig, axs.flatten()[i], Mstar_cent=Mstar_cent, Mstar_binwidth=Mstar_binwidth)
    ax.set_ylim(1.1e0, 9.9e7)

handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, 1.0), fontsize='medium', title_fontsize='medium',
           title=r'TNG-Cluster $z=0$ Satellites $M_{\rm gas}^{\rm sat} > 10^9\, {\rm M_\odot},\ M_{\rm 200c}^{\rm host} \sim 10^{15}\, {\rm M_\odot}$')
axs[0].set_ylabel(r'Gas Densitity [${\rm M_\odot\, kpc^{-3}}$]', fontsize='small')
if savefig:
    fname = 'L680n8192TNG_SGRP_narrowbin_multipanel.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')


In [ ]:
def add_quartiles_axis(ax, grp_dict_keys, dset, 
                       dset_key='SubhaloGasMassShells', time='z0'):
    """
    given the set of grp_dict_keys of interest, plot the profiles of the top 
    and bottom 25th percentiles based on the dset
    """

    mask = dset > 0
    dset = np.log10(dset[mask])

    grp_dict_keys_bin = grp_dict_keys[mask]

    bins = [np.percentile(dset, 0),
            np.percentile(dset, 25),
            np.percentile(dset, 50),
            np.percentile(dset, 75),
            np.percentile(dset, 100)*1.01]

    for i, bin in enumerate(bins):
        if i == len(bins) - 1:
            break

        if i == 1 or i == 2:
            continue

        bin_lo = bins[i]
        bin_hi = bins[i+1]
        indices = (dset >= bin_lo) & (dset < bin_hi)
        keys = grp_dict_keys_bin[indices]

        stacked_dict = return_stacked_dict(keys, dset_key=dset_key, time=time)

        vals_50 = stacked_dict['50_norm']
        mask = vals_50 > 1

        color = cmap(norm(10.**(np.median(dset[indices]))))

        ax.plot(radii_norm_bincents[mask], vals_50[mask], '-', color=color, linewidth=4,
                path_effects=[pe.Stroke(linewidth=7, foreground='k'), pe.Normal()])

    return ax


In [ ]:
Sim = TNGCluster_all_gasz0
tau_dict = Sim.tau_dict

Mstarz0 = tau_dict[Mstarz0_key]
M200cz0 = tau_dict[M200cz0_key]

fig, axs = plt.subplots(2, 2, figsize=(figsizewidth, figsizeheight*1.1))

Mstar_binedges = [10.**(10.0), 10.**(10.099)]
M200c_binedges = [10.**(14.9), 10.**(15.1)]

mask = ((Mstarz0 > Mstar_binedges[0]) & (Mstarz0 < Mstar_binedges[1]) & 
        (M200cz0 > M200c_binedges[0]) & (M200cz0 < M200c_binedges[1]))

subfindIDs = tau_dict['SubfindID'][mask]
grp_dict_keys_bin = []
for subfindID in subfindIDs:
    grp_dict_keys_bin.append('%08d'%subfindID)
grp_dict_keys_bin = np.array(grp_dict_keys_bin)

colors_dset_key = 'SubhaloBHMass'
_, norm, phys, color_dset = return_stacked_dict(grp_dict_keys_bin, return_all_profiles=True, return_color_dset=colors_dset_key)
    
xs = []
ys = []
cs = []
for i, _ in enumerate(grp_dict_keys_bin):
    vals = norm[i]
    indices = vals > 1
    xs.append(radii_norm_bincents[indices])
    ys.append(vals[indices])
    cs.append(color_dset[i])

ax = axs[0,1]
ax.set_xscale('log')
ax.set_yscale('log')

cmap = mpl.cm.get_cmap('magma')
norm = mpl.colors.LogNorm(vmin=np.percentile(color_dset, 5), vmax=np.percentile(color_dset, 95))

lc = ru.multiline(xs, ys, cs, ax=ax, linewidths=0.75, cmap=cmap, norm=norm)

# add the colorbar 
loc = 'upper right'
cbar = plt.colorbar(lc, ax=ax, orientation='vertical', extend='both')
cbar.set_label(r'SMBH Mass [$M_{\rm SMBH}^{\rm sat} / {\rm M_\odot}$]', labelpad=5, fontsize='small')

ax.set_xlabel(r'Satellite-Centric Distance [$r/R_{\rm half,\star}$]', fontsize='small')
ax.set_ylabel(r'Gas Density [${\rm M_{\odot}\, kpc^{-3}}$]', fontsize='small')

ax.text(0.95, 0.95, r'$M_{\star}^{\rm sat} \sim 10^{10}\, {\rm M_\odot}$ (%d)'%(color_dset.size),
        ha='right', va='top', transform=ax.transAxes, fontsize='small')

ax = add_quartiles_axis(ax, grp_dict_keys_bin, color_dset)

ax.set_ylim(1.1e0, 9.9e7)
ax.set_xlim(5e-1, 9e1)
ax.axvline(2.0, c='black', ls='--', marker='None', zorder=1)

# now by RP
ax = axs[0,0]
colors_dset_key = 'LBEramPressure'
_, norm, phys, color_dset = return_stacked_dict(grp_dict_keys_bin, return_all_profiles=True, return_color_dset=colors_dset_key)
    
xs = []
ys = []
cs = []
for i, _ in enumerate(grp_dict_keys_bin):
    vals = norm[i]
    indices = vals > 1
    xs.append(radii_norm_bincents[indices])
    ys.append(vals[indices])
    cs.append(color_dset[i])

ax.set_xscale('log')
ax.set_yscale('log')

cmap = mpl.cm.get_cmap('CMRmap')
norm = mpl.colors.LogNorm(vmin=np.percentile(color_dset, 5), vmax=np.percentile(color_dset, 95))

lc = ru.multiline(xs, ys, cs, ax=ax, linewidths=0.75, cmap=cmap, norm=norm)

# add the colorbar 
loc = 'upper right'
cbar = plt.colorbar(lc, ax=ax, orientation='vertical', extend='both')
cbar.set_label(r'Ram pressure [${\rm erg\, cm^{-3}}$]', labelpad=5, fontsize='small')

ax.set_xlabel(r'Satellite-Centric Distance [$r/R_{\rm half,\star}$]', fontsize='small')
ax.set_ylabel(r'Gas Density [${\rm M_{\odot}\, kpc^{-3}}$]', fontsize='small')

ax.text(0.95, 0.95, r'$M_{\star}^{\rm sat} \sim 10^{10}\, {\rm M_\odot}$ (%d)'%(color_dset.size),
        ha='right', va='top', transform=ax.transAxes, fontsize='small')

ax = add_quartiles_axis(ax, grp_dict_keys_bin, color_dset)

ax.set_ylim(1.1e0, 9.9e7)
ax.set_xlim(5e-1, 9e1)
ax.axvline(2.0, c='black', ls='--', marker='None', zorder=1)

# bottom panels
Mstar_binedges = [10.**(10.7), 10.**(10.8)]
M200c_binedges = [10.**(14.9), 10.**(15.1)]

mask = ((Mstarz0 > Mstar_binedges[0]) & (Mstarz0 < Mstar_binedges[1]) & 
        (M200cz0 > M200c_binedges[0]) & (M200cz0 < M200c_binedges[1]))

subfindIDs = tau_dict['SubfindID'][mask]
grp_dict_keys_bin = []
for subfindID in subfindIDs:
    grp_dict_keys_bin.append('%08d'%subfindID)
grp_dict_keys_bin = np.array(grp_dict_keys_bin)

colors_dset_key = 'SubhaloBHMass'
_, norm, phys, color_dset = return_stacked_dict(grp_dict_keys_bin, return_all_profiles=True, return_color_dset=colors_dset_key)
    
xs = []
ys = []
cs = []
for i, _ in enumerate(grp_dict_keys_bin):
    vals = norm[i]
    indices = vals > 1
    xs.append(radii_norm_bincents[indices])
    ys.append(vals[indices])
    cs.append(color_dset[i])

ax = axs[1,1]
ax.set_xscale('log')
ax.set_yscale('log')

cmap = mpl.cm.get_cmap('magma')
norm = mpl.colors.LogNorm(vmin=np.percentile(color_dset, 5), vmax=np.percentile(color_dset, 95))

lc = ru.multiline(xs, ys, cs, ax=ax, linewidths=0.75, cmap=cmap, norm=norm)

# add the colorbar 
loc = 'upper right'
cbar = plt.colorbar(lc, ax=ax, orientation='vertical', extend='both')
cbar.set_label(r'SMBH Mass [$M_{\rm SMBH}^{\rm sat} / {\rm M_\odot}$]', labelpad=5, fontsize='small')

ax.set_xlabel(r'Satellite-Centric Distance [$r/R_{\rm half,\star}$]', fontsize='small')
ax.set_ylabel(r'Gas Density [${\rm M_{\odot}\, kpc^{-3}}$]', fontsize='small')

ax.text(0.95, 0.95, r'$M_{\star}^{\rm sat} \sim 10^{10.75}\, {\rm M_\odot}$ (%d)'%(color_dset.size),
        ha='right', va='top', transform=ax.transAxes, fontsize='small')

ax = add_quartiles_axis(ax, grp_dict_keys_bin, color_dset)

ax.set_ylim(1.1e0, 9.9e7)
ax.set_xlim(5e-1, 9e1)
ax.axvline(2.0, c='black', ls='--', marker='None', zorder=1)

# now by RP
ax = axs[1,0]
colors_dset_key = 'LBEramPressure'
_, norm, phys, color_dset = return_stacked_dict(grp_dict_keys_bin, return_all_profiles=True, return_color_dset=colors_dset_key)
    
xs = []
ys = []
cs = []
for i, _ in enumerate(grp_dict_keys_bin):
    vals = norm[i]
    indices = vals > 1
    xs.append(radii_norm_bincents[indices])
    ys.append(vals[indices])
    cs.append(color_dset[i])

ax.set_xscale('log')
ax.set_yscale('log')

cmap = mpl.cm.get_cmap('CMRmap')
norm = mpl.colors.LogNorm(vmin=np.percentile(color_dset, 5), vmax=np.percentile(color_dset, 95))

lc = ru.multiline(xs, ys, cs, ax=ax, linewidths=0.75, cmap=cmap, norm=norm)

# add the colorbar 
loc = 'upper right'
cbar = plt.colorbar(lc, ax=ax, orientation='vertical', extend='both')
cbar.set_label(r'Ram pressure [${\rm erg\, cm^{-3}}$]', labelpad=5, fontsize='small')

ax.set_xlabel(r'Satellite-Centric Distance [$r/R_{\rm half,\star}$]', fontsize='small')
ax.set_ylabel(r'Gas Density [${\rm M_{\odot}\, kpc^{-3}}$]', fontsize='small')

ax.text(0.95, 0.95, r'$M_{\star}^{\rm sat} \sim 10^{10.75}\, {\rm M_\odot}$ (%d)'%(color_dset.size),
        ha='right', va='top', transform=ax.transAxes, fontsize='small')

ax = add_quartiles_axis(ax, grp_dict_keys_bin, color_dset)

ax.set_ylim(1.1e0, 9.9e7)
ax.set_xlim(5e-1, 9e1)
ax.axvline(2.0, c='black', ls='--', marker='None', zorder=1)

for ax in axs.flatten():  
    ax = add_ISM_CGM_line(ax)

title = (r'TNG-Cluster $z=0$ Satellites with ' +
         r'$M_{\rm gas}^{\rm sat} > 10^9\, {\rm M_\odot},\ M_{\rm 200c}^{\rm host}\sim10^{15}\, {\rm M_\odot}$')
fig.suptitle(title, fontsize='medium')

if savefig:
    fname = 'L680n8192TNG_SGRP_narrowbins_individualcurves_multipanel.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')



In [ ]:
# calculate the CGM mass as the total mass outside 2R_half,star
# we want the CGM mass at infall for all z=0 satellites, but the CGM 
# mass at z=0 only for the gaseous satellites. then we'll manually place the 
# the CGM masses of the gas-poor satellites as a random number between 10^6-7 Msun

# find the grp_dict_keys where tau_100^AllGas is reached
Sim = TNGCluster_all
tau_dict = Sim.tau_dict

SubhaloCGMMass_z0 = np.zeros(len(grp_dict_keys), dtype=float) - 1
SubhaloCGMMass_infall = np.zeros(len(grp_dict_keys), dtype=float) - 1
Mstar_z0 = tau_dict[Mstarz0_key]

for i, grp_dict_key in enumerate(grp_dict_keys):
    group = grp_dict[grp_dict_key]

    radii_norm_z0 = group['radii'][0,:] / group['Subhalo_Rgal'][0]
    gasmass_shells_z0 = group['SubhaloGasMassShells'][0,:]
    SubhaloCGMMass_z0[i] = np.sum(gasmass_shells_z0[radii_norm_z0 >= 1])

    tau_infall = group['tau_infall_Gas']
    time_index = np.where(tau_infall == 0)[0][0]
    radii_norm_infall = group['radii'][time_index,:] / group['Subhalo_Rgal'][time_index]
    gasmass_shells_infall = group['SubhaloGasMassShells'][time_index,:]
    SubhaloCGMMass_infall[i] = np.sum(gasmass_shells_infall[radii_norm_infall >= 1])   

np.random.seed(1)
mask = SubhaloCGMMass_z0 <= 0
CGMmass_lo_log = 7.0
CGMmass_low_width_log = 1.0
SubhaloCGMMass_z0[mask] = 10.**(np.random.uniform(CGMmass_lo_log, CGMmass_lo_log+CGMmass_low_width_log, SubhaloCGMMass_z0[mask].size))

In [ ]:
fig, ax = plt.subplots(figsize=(figsizewidth_column*1.5, figsizeheight_column*1.5))
indices1 = ((Mstar_z0 > 0) &
            (SubhaloCGMMass_infall > 0))

indices2 = ((Mstar_z0 > 0) &
            (SubhaloCGMMass_z0 > 10.**(CGMmass_lo_log+CGMmass_low_width_log)))

x1 = np.log10(Mstar_z0[indices1])
y1 = np.log10(SubhaloCGMMass_infall[indices1])
x2 = np.log10(Mstar_z0[indices2])
y2 = np.log10(SubhaloCGMMass_z0[indices2])

himstar_indices1 = x1 > 12.25
himstar_indices2 = x2 > 12.25

mass_binwidth = 0.125

percentiles = [16, 50, 84]
sigmas = [1]
stats = ru.return2dhiststats_dict(x1[~himstar_indices1], y1[~himstar_indices1], mass_binwidth, percentiles)
bin_cents = 10**(stats['bin_cents'])

for i, sigma in enumerate(sigmas):
    lo_percentile = percentiles[i]
    hi_percentile = percentiles[-(i+1)]
    
    y_los = 10.**(stats[lo_percentile])
    y_his = 10.**(stats[hi_percentile])
    
    ax.fill_between(bin_cents, y_los, y_his, color='black', alpha=1./(sigma+3.))

y_50s = 10.**(stats[50])
ax.plot(bin_cents, y_50s, c='black', ls='-', lw=4, marker='None', label='at infall as centrals (%d)'%(indices1[indices1].size))
ax.plot(10.**(x1[himstar_indices1]), 10.**(y1[himstar_indices1]), marker='o', ls='None', c='black', ms=3, fillstyle='none', mew=1)
ax.plot([10.**(np.median(x1[himstar_indices1])+.0125)],[10.**(np.median(y1[himstar_indices1]))],
        marker='s', fillstyle='none', ms=12, mew=2, c='black')
ax.hlines(10.**(np.median(y1[himstar_indices1])), 10.**(np.percentile(x1[himstar_indices1], 16)), 10.**(np.percentile(x1[himstar_indices1], 84)), ls='-', lw=2, color='black')
ax.vlines(10.**(np.median(x1[himstar_indices1]+.0125)), 10.**(np.percentile(y1[himstar_indices1], 16)), 10.**(np.percentile(y1[himstar_indices1], 84)), ls='-', lw=2, color='black')



stats = ru.return2dhiststats_dict(x2[~himstar_indices2], y2[~himstar_indices2], mass_binwidth, percentiles)
bin_cents = 10**(stats['bin_cents'])

for i, sigma in enumerate(sigmas):
    lo_percentile = percentiles[i]
    hi_percentile = percentiles[-(i+1)]
    
    y_los = 10.**(stats[lo_percentile])
    y_his = 10.**(stats[hi_percentile])
    
    ax.fill_between(bin_cents, y_los, y_his, color='tab:purple', alpha=1./(sigma+3.))

y_50s = 10.**(stats[50])
ax.plot(bin_cents, y_50s, c='tab:pink', ls='-', lw=4, marker='None', label=r'at $z=0$ as satellites (%d)'%(indices2[indices2].size))
ax.plot(10.**(x2[himstar_indices2]), 10.**(y2[himstar_indices2]), marker='o', ls='None', c='tab:pink', ms=3, fillstyle='none', mew=1)
ax.plot([10.**(np.median(x2[himstar_indices2]-.0125))],[10.**(np.median(y2[himstar_indices2]))],
        marker='s', fillstyle='none', ms=12, mew=2, c='tab:pink')
ax.hlines(10.**(np.median(y2[himstar_indices2])), 10.**(np.percentile(x2[himstar_indices2], 16)), 10.**(np.percentile(x2[himstar_indices2], 84)), ls='-', lw=2, color='tab:pink')
ax.vlines(10.**(np.median(x2[himstar_indices2]-0.0125)), 10.**(np.percentile(y2[himstar_indices2], 16)), 10.**(np.percentile(y2[himstar_indices2], 84)), ls='-', lw=2, color='tab:pink')

ax.plot(Mstar_z0[mask], SubhaloCGMMass_z0[mask], markevery=1, color='tab:pink', ms=1, fillstyle='none', mew=0.1, rasterized=True)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'Galaxy Stellar Mass [$M_\star^{\rm gal} / {\rm M_\odot}$]')
ax.set_ylabel(r'Galaxy CGM Mass [$M_{\rm CGM}^{\rm gal} / {\rm M_\odot}$]')
ax.legend(loc='upper left')

text = (r'\begin{align*}' + 
        #r'& M_{\rm gas}^{\rm sat} > 10^9\, {\rm M_\odot} \\' + 
        r'& M_{\star}^{\rm sat} \in (10^{10},\, 10^{12.5})\, {\rm M_\odot} \\' + 
        r'& M_{\rm 200c}^{\rm host} \in (10^{14.3},\, 10^{15.4})\, {\rm M_\odot} \\' + 
        #r'& %d\ {\rm galaxies}'%(indices2[indices2].size) + 
        r'\end{align*}')
ax.text(0.02, 0.81, text, ha='left', va='top', transform=ax.transAxes)

text = ax.text(0.02, 0.2, r'$M_{\rm CGM}^{\rm sat}(z=0) < 10^9\, {\rm M_\odot}\ (%d)$'%(mask[mask].size), ha='left', va='bottom', transform=ax.transAxes)
#text.set_path_effects([pe.Stroke(foreground='white', linewidth=0.5), pe.Normal()])

if savefig:
    fname = 'L680n8192TNG_all_gasz0_SCGM_Mstarz0.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')


In [ ]:

fig, ax = plt.subplots(figsize=(figsizewidth_column*1.5, figsizeheight_column*1.5))
indices = ((Mstar_z0 > 0) &
           (SubhaloCGMMass_infall > 0) & 
           (SubhaloCGMMass_z0 > 0))

x = np.log10(Mstar_z0[indices])
y1 = np.log10(SubhaloCGMMass_infall[indices])
y2 = np.log10(SubhaloCGMMass_z0[indices])

himstar_indices = x > 11.5

mass_binwidth = 0.1

percentiles = [16, 50, 84]
sigmas = [1]
stats = ru.return2dhiststats_dict(x[~himstar_indices], y1[~himstar_indices], mass_binwidth, percentiles)
bin_cents = 10**(stats['bin_cents'])

for i, sigma in enumerate(sigmas):
    lo_percentile = percentiles[i]
    hi_percentile = percentiles[-(i+1)]
    
    y_los = 10.**(stats[lo_percentile])
    y_his = 10.**(stats[hi_percentile])
    
    ax.fill_between(bin_cents, y_los, y_his, color='black', alpha=1./(sigma+3.))

y_50s = 10.**(stats[50])
ax.plot(bin_cents, y_50s, c='black', ls='-', lw=4, marker='None', label='at infall as centrals')
ax.plot(10.**(x[himstar_indices]), 10.**(y1[himstar_indices]), marker='o', ls='None', c='black', ms=3, fillstyle='none', mew=1)
ax.plot([10.**(np.median(x[himstar_indices])+.0125)],[10.**(np.median(y1[himstar_indices]))],
        marker='s', fillstyle='none', ms=12, mew=2, c='black')
ax.hlines(10.**(np.median(y1[himstar_indices])), 10.**(np.percentile(x[himstar_indices], 16)), 10.**(np.percentile(x[himstar_indices], 84)), ls='-', lw=2, color='black')
ax.vlines(10.**(np.median(x[himstar_indices]+.0125)), 10.**(np.percentile(y1[himstar_indices], 16)), 10.**(np.percentile(y1[himstar_indices], 84)), ls='-', lw=2, color='black')

stats = ru.return2dhiststats_dict(x[~himstar_indices], y2[~himstar_indices], mass_binwidth, percentiles)
bin_cents = 10**(stats['bin_cents'])

for i, sigma in enumerate(sigmas):
    lo_percentile = percentiles[i]
    hi_percentile = percentiles[-(i+1)]
    
    y_los = 10.**(stats[lo_percentile])
    y_his = 10.**(stats[hi_percentile])
    
    ax.fill_between(bin_cents, y_los, y_his, color='tab:purple', alpha=1./(sigma+3.))

y_50s = 10.**(stats[50])
ax.plot(bin_cents, y_50s, c='tab:pink', ls='-', lw=4, marker='None', label=r'at $z=0$ as satellites')
ax.plot(10.**(x[himstar_indices]), 10.**(y2[himstar_indices]), marker='o', ls='None', c='tab:pink', ms=3, fillstyle='none', mew=1)
ax.plot([10.**(np.median(x[himstar_indices]-.0125))],[10.**(np.median(y2[himstar_indices]))],
        marker='s', fillstyle='none', ms=12, mew=2, c='tab:pink')
ax.hlines(10.**(np.median(y2[himstar_indices])), 10.**(np.percentile(x[himstar_indices], 16)), 10.**(np.percentile(x[himstar_indices], 84)), ls='-', lw=2, color='tab:pink')
ax.vlines(10.**(np.median(x[himstar_indices]-0.0125)), 10.**(np.percentile(y2[himstar_indices], 16)), 10.**(np.percentile(y2[himstar_indices], 84)), ls='-', lw=2, color='tab:pink')
"""
ax.errorbar([10.**(np.median(x[himstar_indices]))],[10.**(np.median(y2[himstar_indices]))],
            xerr=[10.**(np.median(x[himstar_indices]))-10.**(np.min(x[himstar_indices])), -10.**(np.median(x[himstar_indices]))+10.**(np.max(x[himstar_indices]))],
            yerr=[10.**(np.median(y2[himstar_indices]))-10.**(np.min(y2[himstar_indices])), -10.**(np.median(y2[himstar_indices]))+10.**(np.max(y2[himstar_indices]))],
            marker='s', fillstyle='none', ms=12, mew=2, c='tab:pink')
"""
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'Galaxy Stellar Mass [$M_\star^{\rm gal} / {\rm M_\odot}$]')
ax.set_ylabel(r'Galaxy CGM Mass [$M_{\rm CGM}^{\rm gal} / {\rm M_\odot}$]')
ax.legend(loc='upper left')

text = (r'\begin{align*}' + 
        r'& M_{\rm gas}^{\rm sat} > 10^9\, {\rm M_\odot} \\' + 
        r'& M_{\star}^{\rm sat} \in (10^{10},\, 10^{11.8})\, {\rm M_\odot} \\' + 
        r'& M_{\rm 200c}^{\rm host} \in (10^{14.4},\, 10^{15.4})\, {\rm M_\odot} \\' + 
        r'& %d\ {\rm galaxies}'%(indices.size) + 
        r'\end{align*}')
ax.text(0.02, 0.815, text, ha='left', va='top', transform=ax.transAxes)

savefig=False
if savefig:
    fname = 'L680n8192TNG_all_gasz0_SCGM_Mstarz0.pdf'
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')


In [ ]:
rmin = 1
rmax = 1.0e3
binwidth_log = 0.1
_, radii_phys_bincents = ru.returnlogbins([rmin, rmax], binwidth_log)
radii_norm_bincents = grp_dict[grp_dict_keys_gasz0[0]]['radii'][0,:] / grp_dict[grp_dict_keys_gasz0[0]]['Subhalo_Rgal'][0]

def return_stacked_dict(tau_dict, dset_key, dset_bins):
    """
    Given the tau_dict, dset_key, and dset_dset_bins, bin all subhalos within
    the tau_dict by the dset_key and dset_dset_bins. Then within the stack,
    calculate the medians, 16th, and 84th percentiles of the densities both
    in normalized and physical radial coordinates. Note that this assumes linear
    interpolation between the nonzero density entries. 
    Resulting dictionary takes indices as keys, which lead to another dictionary 
    that takes '50_phys', '16_phys', '84_phys', '50_norm', '16_norm', '84_norm',
    and 'Ngal'. 'Ngal' is an integer, and all other entries are 1D arrays 
    corresponding to the global radii_phys_bincents and radii_norm_bincents 1D arrays.
    """
    
    result_dict = {}
    
    for bin_i in range(len(dset_bins)-1):
        result_dict[bin_i] = {}
        indices = ((tau_dict[dset_key] > dset_bins[bin_i]) &
                   (tau_dict[dset_key] < dset_bins[bin_i+1]))

        # initalize the outputs
        result_dict[bin_i]['50_phys'] = np.zeros((len(indices[indices]), len(radii_phys_bincents)), dtype=float) - 1.
        result_dict[bin_i]['16_phys'] = result_dict[bin_i]['50_phys'].copy()
        result_dict[bin_i]['84_phys'] = result_dict[bin_i]['50_phys'].copy()
        result_dict[bin_i]['50_norm'] = np.zeros((len(indices[indices]), len(radii_norm_bincents)), dtype=float) - 1.
        result_dict[bin_i]['16_norm'] = result_dict[bin_i]['50_norm'].copy()
        result_dict[bin_i]['84_norm'] = result_dict[bin_i]['50_norm'].copy()

        phys = np.zeros((len(indices[indices]), len(radii_phys_bincents)), dtype=float) - 1.
        norm = np.zeros((len(indices[indices]), len(radii_norm_bincents)), dtype=float) - 1.

        for index, subfindID in enumerate(tau_dict['SubfindID'][indices]):
            key = '%08d'%subfindID
            group = grp_dict[key]
            radii = group['radii'][0,:].copy()
            rgal = group['Subhalo_Rgal'][0].copy()
            densities = group['SubhaloDensityShells'][0,:].copy()
            non0_indices = densities > 0

            # interpolate the non0 densities and radii at the bincents
            phys[index,:] = np.interp(radii_phys_bincents, radii[non0_indices], densities[non0_indices])
            norm[index,:] = np.interp(radii_norm_bincents, radii[non0_indices]/rgal, densities[non0_indices])

        # finish loop of indices, save final results
        if np.min(phys) == -1:
            raise ValueError('not all indices of densities have been assigned.')

        result_dict[bin_i]['50_phys'] = np.median(phys, axis=0)
        result_dict[bin_i]['16_phys'] = np.percentile(phys, 16, axis=0)
        result_dict[bin_i]['84_phys'] = np.percentile(phys, 84, axis=0)

        result_dict[bin_i]['50_norm'] = np.median(norm, axis=0)
        result_dict[bin_i]['16_norm'] = np.percentile(norm, 16, axis=0)
        result_dict[bin_i]['84_norm'] = np.percentile(norm, 84, axis=0)

        result_dict[bin_i]['Ngal'] = len(norm)
        
    # finish loop over bins. return to main
    return result_dict


def plot_stacked_dict(Sim, stacked_dict, dset_key, dset_bins, dset_label, colors, radii='norm', savefig=False):
    """
    Plot the stacked_dict given the dict, dset_bins, and label.
    If radii=='norm', then use the normalized radial coordinates.
    If radii=='phys', use physical radial coordinates.
    """
    if radii == 'norm':
        x_label = r'Distance [$ r / 2R_{\rm half}^\star$]'
        radii_bincents = radii_norm_bincents
    elif radii == 'phys':
        x_label = r'Distance [$r / {\rm kpc}$]'
        radii_bincents = radii_phys_bincents
    else:
        raise ValueError('invalid radii value %s. Accepted values are norm and phys'%radii)
        
    if len(colors) < len(stacked_dict.keys()):
        raise ValueError('too few colors given')
    
    fig, ax = plt.subplots()
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(x_label)
    ax.set_ylabel(r'Gas Densitity [$\rho_{\rm Gas}^{\rm sat}({\rm shells}) / {\rm M_\odot\, kpc^{-3}}$]' )

    for i, bin_key in enumerate(stacked_dict.keys()):
        color = colors[i]
        densities_50 = stacked_dict[bin_key]['50_%s'%radii]
        densities_16 = stacked_dict[bin_key]['16_%s'%radii]
        densities_84 = stacked_dict[bin_key]['84_%s'%radii]
        Ngal = stacked_dict[bin_key]['Ngal']
        ax.plot(radii_bincents, densities_50, '-', color=color, label=r'$[%.2g, %.2g)\ (%d)$'%(dset_bins[i], dset_bins[i+1], Ngal))
        ax.fill_between(radii_bincents, densities_16, densities_84, color=color, alpha=0.2)

    ax.legend(title=dset_label)
    ax.set_title(Sim.plot_label)
    
    if (savefig):
        fname = '%s_MGasRadProf_Stack-%s_%s.pdf'%(Sim.save_label, dset_key, radii)
        for direc in direc_list:
            fig.savefig(direc + fname, bbox_inches='tight')
    
    return fig, ax


In [ ]:
tau_dict = TNGCluster_all_gasz0.tau_dict
dset_key = Mstarz0_key
dset_bins = [10.**(10.0), 10.**(10.5), 10.**(11.0), 10.**(12.7)]
dset_label = r'$M_\star^{\rm sat} / {\rm M_\odot} \in$'
colors = ['lightskyblue', 'dodgerblue', 'darkblue']

stacked_dict = return_stacked_dict(tau_dict, dset_key, dset_bins)

fig, ax = plot_stacked_dict(TNGCluster_all_gasz0, stacked_dict, dset_key, dset_bins, dset_label, colors=colors, radii='norm', savefig=savefig)

fig, ax = plot_stacked_dict(TNGCluster_all_gasz0, stacked_dict, dset_key, dset_bins, dset_label, colors=colors, radii='phys', savefig=savefig)

In [ ]:
dset_key = MHGasz0_key
dset_bins = [10.**(10.0), 10.**(11.0), 10.**(12.0), 10.**(14.5)]
dset_label = r'$M_{\rm HotGas}^{\rm sat} / {\rm M_\odot} \in$'
colors = ['lightsalmon', 'red', 'darkred']

stacked_dict = return_stacked_dict(tau_dict, dset_key, dset_bins)

fig, ax = plot_stacked_dict(TNGCluster_all_gasz0, stacked_dict, dset_key, dset_bins, dset_label, colors=colors, radii='norm', savefig=savefig)

fig, ax = plot_stacked_dict(TNGCluster_all_gasz0, stacked_dict, dset_key, dset_bins, dset_label, colors=colors, radii='phys', savefig=savefig)

In [ ]:
dset_key = HCDnz0_key
dset_bins = [0.0, 1.0, 2.0, 4.0]
dset_label = r'$d_{\rm sat}^{\rm host} / R_{\rm 200c}^{\rm host} \in$'
colors = ['violet', 'mediumpurple', 'indigo']

stacked_dict = return_stacked_dict(tau_dict, dset_key, dset_bins)

fig, ax = plot_stacked_dict(TNGCluster_all_gasz0, stacked_dict, dset_key, dset_bins, dset_label, colors=colors, radii='norm', savefig=savefig)

fig, ax = plot_stacked_dict(TNGCluster_all_gasz0, stacked_dict, dset_key, dset_bins, dset_label, colors=colors, radii='phys', savefig=savefig)

In [ ]:
dset_key = time_since_infall_key
dset_bins = [0.0, 1.0, 2.0, 6.0]
dset_label = r'time since infall [Gyr] $\in$'
colors = ['darkgoldenrod', 'orange', 'bisque']

stacked_dict = return_stacked_dict(tau_dict, dset_key, dset_bins)

fig, ax = plot_stacked_dict(TNGCluster_all_gasz0, stacked_dict, dset_key, dset_bins, dset_label, colors=colors, radii='norm', savefig=savefig)

fig, ax = plot_stacked_dict(TNGCluster_all_gasz0, stacked_dict, dset_key, dset_bins, dset_label, colors=colors, radii='phys', savefig=savefig)

In [ ]:
Sim = TNGCluster_all

tau_dict = Sim.tau_dict
Mstar = tau_dict[Mstarz0_key].copy()
MHGas = tau_dict[MHGasz0_key].copy()

MHGas_lolim = 1.0e7
MHGas[MHGas < MHGas_lolim] = MHGas_lolim

indices = Mstar > 0

mass_binwidth = 0.1

percentiles = [0.15, 2.5, 16, 50, 84, 97.5, 99.85]
sigmas = [3, 2, 1]
stats = ru.return2dhiststats_dict(np.log10(Mstar[indices]), np.log10(MHGas[indices]), mass_binwidth, percentiles)
bin_cents = 10**(stats['bin_cents'])

fig, ax = plt.subplots()

for i, sigma in enumerate(sigmas):
    lo_percentile = percentiles[i]
    hi_percentile = percentiles[-(i+1)]
    
    y_los = 10.**(stats[lo_percentile])
    y_his = 10.**(stats[hi_percentile])
    
    ax.fill_between(bin_cents, y_los, y_his, color=Sim.plotkwargs['color'], alpha=1./(sigma+1.), label='%d'%sigma + r'$\sigma$')

y_50s = 10.**(stats[50])
ax.plot(bin_cents, y_50s, c='dimgray', ls='-', lw=4, marker='None', label='Median')
  

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title(Sim.plot_label)
ax.set_xlabel(Mstarz0_label)
ax.set_ylabel(MHGasz0_label)
h_orig, l_orig = ax.get_legend_handles_labels()
h_new = [h_orig[0]]
l_new = [l_orig[0]]
for index in range(1, len(h_orig)):
    h_new.append(h_orig[-index])
    l_new.append(l_orig[-index])
    print()
ax.legend(h_new, l_new)
if savefig:
    fname = '%s_%s-%s.pdf'%(Sim.save_label, MHGasz0_key, Mstarz0_key)
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')

In [ ]:
Sim = TNGCluster_all_z0
mass_binwidth = 0.15

tau_dict = Sim.tau_dict
Mstar = tau_dict[Mstarz0_key].copy()
MHGas = tau_dict[MHGasz0_key].copy()
MGas = tau_dict[MGasz0_key].copy()
MCGas = tau_dict[MCGasz0_key].copy()

Mstar_bins, Mstar_bincents = ru.returnlogbins(Mstar, mass_binwidth)

sort_indices = np.digitize(Mstar, Mstar_bins)
gaslessMgas_frac = np.zeros(Mstar_bincents.size, Mstar_bincents.dtype) - 1
NgaslessMgas = gaslessMgas_frac.copy()
gaslessMgas90_frac = gaslessMgas_frac.copy()
gaslessMgas100_frac = gaslessMgas_frac.copy()
gaslessMHGas_frac = gaslessMgas_frac.copy()
NgaslessMHgas = gaslessMgas_frac.copy()
gaslessMCGas_frac = gaslessMgas_frac.copy()
NgaslessMCgas = gaslessMgas_frac.copy()

for i, bincent in enumerate(Mstar_bincents):
    subset_indices = sort_indices == (i+1)
    Mgas_subset = MGas[subset_indices]
    if Mgas_subset.size == 0:
        continue
    gaslessMgas_frac[i] = Mgas_subset[Mgas_subset != 0].size / Mgas_subset.size 
    NgaslessMgas[i] = Mgas_subset[Mgas_subset != 0].size 
    gaslessMgas90_frac[i] = Mgas_subset[Mgas_subset >= 1.0e9].size / Mgas_subset.size
    gaslessMgas100_frac[i] = Mgas_subset[Mgas_subset >= 1.0e10].size / Mgas_subset.size
    Mhgas_susbset = MHGas[subset_indices]
    gaslessMHGas_frac[i] = Mhgas_susbset[Mhgas_susbset != 0].size / Mhgas_susbset.size
    NgaslessMHgas[i] = Mhgas_susbset[Mhgas_susbset != 0].size 
    Mcgas_subset = MCGas[subset_indices]
    gaslessMCGas_frac[i] = Mcgas_subset[Mcgas_subset != 0].size / Mcgas_subset.size
    NgaslessMCgas[i] = Mcgas_subset[Mcgas_subset != 0].size 



In [ ]:
indices = gaslessMgas_frac != -1
fig, axs = plt.subplots(2, 1, figsize=(figsizewidth_column, figsizeheight_column * 2))

# top axis: fraction of satellites
ax = axs[0]
ax.plot(Mstar_bincents[indices], gaslessMgas_frac[indices], color='tab:purple', marker='None', ls='-', lw=4, label=r'$M_{\rm AllGas}^{\rm sat} > 10^{9}\, {\rm M_\odot}$')
ax.plot(Mstar_bincents[indices], gaslessMHGas_frac[indices], color='tab:red', marker='None', ls='--', label=r'$M_{\rm HotGas}^{\rm sat} > 10^{9}\, {\rm M_\odot}$')
ax.plot(Mstar_bincents[indices], gaslessMCGas_frac[indices], color='tab:blue', marker='None', ls='-.', label=r'$M_{\rm ColdGas}^{\rm sat} > 10^{9}\, {\rm M_\odot}$')
ax.set_xscale('log')
ax.set_title('TNG-Cluster All (%d)'%Mstar.size)
ax.set_xlabel(Mstarz0_label)
ax.set_ylabel('Fraction of Satellites')
ax.legend(loc='upper left', title='Satellites with')

# bottom axis: number of satellites
ax = axs[1]
ax.plot(Mstar_bincents[indices], NgaslessMgas[indices], color='tab:purple', marker='None', ls='-', ds='steps-mid', lw=3)
ax.fill_between(Mstar_bincents[indices], NgaslessMgas[indices], color='tab:purple',step='mid', alpha=0.7)
ax.plot(Mstar_bincents[indices], NgaslessMHgas[indices], color='tab:red', marker='None', ls='--', ds='steps-mid', lw=2)
ax.fill_between(Mstar_bincents[indices], NgaslessMHgas[indices], color='tab:red',step='mid', hatch='//', facecolor='none')
ax.plot(Mstar_bincents[indices], NgaslessMCgas[indices], color='tab:blue', marker='None', ls='--', ds='steps-mid', lw=2)
ax.fill_between(Mstar_bincents[indices], NgaslessMCgas[indices], color='tab:blue',step='mid', hatch='\\\\', facecolor='none')
ax.set_yscale('log')
ax.set_ylabel(r'Number of Satellites')
ax.set_xlabel(Mstarz0_label)
ax.set_xscale('log')
#ax.legend(loc='lower left', title='Satellites with')
savefig=False
if savefig:
    fname = '%s_fgassat-mstarz0.pdf'%(Sim.save_label)
    for direc in direc_list:
        fig.savefig(direc + fname, bbox_inches='tight')

In [ ]:
### get some statistics on the median CGM gas cell size
Sim = TNGCluster_all_gasz0
tau_dict = Sim.tau_dict
SubfindIDs = tau_dict['SubfindID']

basePath = ru.ret_basePath(sim)
snapNum = 99

header  = il.groupcat.loadHeader(basePath, snapNum)
h = 0.6774
a = header['Time'] # scale factor
boxsize = header['BoxSize'] * a / h

gas_hsml_fact = 1.5


Nsubhalos = 500
CGMsizes_medians = np.zeros(Nsubhalos, dtype=float) - 1
CGMsizes_10s = CGMsizes_medians.copy()
CGMsizes_90s = CGMsizes_medians.copy()
NCGM_cells = np.zeros(Nsubhalos, dtype=int) - 1

gas_fields = ['Coordinates', 'Masses', 'Density',
              'InternalEnergy', 'ElectronAbundance', 'StarFormationRate']

for i, subfindID in enumerate(SubfindIDs[:Nsubhalos]):
    gas_cells = il.snapshot.loadSubhalo(basePath, snapNum, subfindID, 'gas', fields=gas_fields)
    gas_cells = ru.calc_temp_dict(gas_cells)
    subhalo = il.snapshot.loadSingle(basePath, snapNum, subhaloID=subfindID) 
    SubhaloPos = subhalo['SubhaloPos'] * a / h
    subhalo_rgal = 2. * subhalo['SubhaloHalfmassRadType'][star_ptn] * a / h

    gas_coordinates = gas_cells['Coordinates'] * a / h

    gas_radii = ru.mag(gas_coordinates, SubhaloPos, boxsize)

    mask = gas_radii >= subhalo_rgal

    if mask[mask].size == 0:
        continue

    Masses = gas_cells['Masses'][mask] * 1.0e10 / h
    Coordinates = gas_cells['Coordinates'][mask] * a / h
    Densities = gas_cells['Density'][mask] * 1.0e10 / h / (a / h)**3
    Sizes = (Masses / (Densities * 4./3. * np.pi))**(1./3.) * gas_hsml_fact

    CGMsizes_medians[i] = np.median(Sizes)
    CGMsizes_10s[i] = np.percentile(Sizes, 10)
    CGMsizes_90s[i] = np.percentile(Sizes, 90)
    NCGM_cells[i] = Sizes.size



In [ ]:
mask = NCGM_cells > 0

fig, ax = plt.subplots()
ax.hist(CGMsizes_medians[mask])

fig, ax = plt.subplots()
ax.hist(np.log10(NCGM_cells[mask] + 1))

print(np.median(CGMsizes_medians[mask]), np.median(CGMsizes_10s[mask]), np.median(CGMsizes_90s[mask]))